In [1]:
import pandas as pd
import re

In [2]:
material_type = ["06"]

In [3]:
mm = pd.read_csv("Vw_MaterialMaster.csv", encoding="utf-8")

# แสดงข้อมูล 5 แถวแรก
print(mm.head())

   Material_Type Material_Number                      Material_Description  \
0            3.0      7803141642          SW,PROXI,3RG4014-0AG30-Z,SIEMENS   
1           32.0      7832234005  FELT,PICK UP PM#4,5,23.00X3.95 M,ATROVIT   
2            3.0     75030202210            CONTACTOR,CA 501, 1NC,ABB  [5]   
3            3.0     75030202220            CONTACTOR,CA 510, 1NO,ABB  [5]   
4            3.0     75030202260      CONTACTOR,CAL 511,1NO 1NC.,ABB  [ZN]   

  PlantID Plant Plant_Description UOM  Moving_avg_price  \
0   PT014  7811      TPC-Ban Pong  PC           5400.00   
1   PT014  7811      TPC-Ban Pong  PC         314484.73   
2   PT011  7560    SKIC-Wangsa-la  ST            124.00   
3   PT011  7560    SKIC-Wangsa-la  ST            127.00   
4   PT011  7560    SKIC-Wangsa-la  ST            250.00   

   Planned_delivery_time  Reorder_point  ...  CompID CompanyCode  \
0                    NaN            2.0  ...   CP005         780   
1                    NaN            0.0 

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_56111/2540408686.py:1: DtypeWarning: Columns (4,11,14,15,16,17,18,19,20,26,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  mm = pd.read_csv("Vw_MaterialMaster.csv", encoding="utf-8")


In [4]:
mm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310160 entries, 0 to 310159
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Material_Type          310159 non-null  float64
 1   Material_Number        310160 non-null  object 
 2   Material_Description   310160 non-null  object 
 3   PlantID                310160 non-null  object 
 4   Plant                  310160 non-null  object 
 5   Plant_Description      310160 non-null  object 
 6   UOM                    310020 non-null  object 
 7   Moving_avg_price       310153 non-null  float64
 8   Planned_delivery_time  240068 non-null  float64
 9   Reorder_point          310160 non-null  float64
 10  Max_Stock_Level        310160 non-null  float64
 11  Storage_Bin            90218 non-null   object 
 12  MRP_Type               175183 non-null  object 
 13  Material_Group         310157 non-null  object 
 14  Purchasing_Group       291806 non-nu

In [5]:
mm["Material_Number"].value_counts

<bound method IndexOpsMixin.value_counts of 0          7803141642
1          7832234005
2         75030202210
3         75030202220
4         75030202260
             ...     
310155    RE377000008
310156    RE377000008
310157    RE377000007
310158    RE377000007
310159    RE377000007
Name: Material_Number, Length: 310160, dtype: object>

In [6]:
mm["Material_Number"].duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
310155     True
310156     True
310157     True
310158     True
310159     True
Name: Material_Number, Length: 310160, dtype: bool

In [7]:
mm["pk_plant_matnum"] = mm["Plant"].astype(str) + "_" + mm["Material_Number"].astype(str)
mm["pk_plant_matnum"].value_counts()

pk_plant_matnum
PO21_PO3315510124          14
PO11_PO3315510121          12
PO21_PO3315510121          12
PO21_PO3315510122          11
OL11_W0OFFICOMPNOTB0003     7
                           ..
LX11_LX32218-1681           1
LX11_LX32218-1702           1
LX11_LX32218-1703           1
LX11_LX32218-1705           1
7811_78107400015            1
Name: count, Length: 293883, dtype: int64

In [8]:
mm.head()

,Material_Type,Material_Number,Material_Description,PlantID,Plant,Plant_Description,UOM,Moving_avg_price,Planned_delivery_time,Reorder_point,...,CompanyCode,CompanyDesc,Consigment_Onhand,Consigment_Blocked,MaterialConsigment,Currency,flagdeleted,createdatetime,updatedatetime,pk_plant_matnum
0,3.0,7803141642,"SW,PROXI,3RG4014-0AG30-Z,SIEMENS",PT014,7811,TPC-Ban Pong,PC,5400.00,NaN,2.0,...,780,"THAI PAPER CO.,LTD.",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:12:55.877,7811_7803141642
1,32.0,7832234005,"FELT,PICK UP PM#4,5,23.00X3.95 M,ATROVIT",PT014,7811,TPC-Ban Pong,PC,314484.73,NaN,0.0,...,780,"THAI PAPER CO.,LTD.",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:28:49.973,7811_7832234005
2,3.0,75030202210,"CONTACTOR,CA 501, 1NC,ABB [5]",PT011,7560,SKIC-Wangsa-la,ST,124.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:31:24.79,7560_75030202210
3,3.0,75030202220,"CONTACTOR,CA 510, 1NO,ABB [5]",PT011,7560,SKIC-Wangsa-la,ST,127.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:53:51.327,7560_75030202220
4,3.0,75030202260,"CONTACTOR,CAL 511,1NO 1NC.,ABB [ZN]",PT011,7560,SKIC-Wangsa-la,ST,250.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 12:02:03.967,7560_75030202260


In [9]:
select_mm_matType = mm[
    mm["Material_Type"]
    .fillna(-1)
    .astype(int)
    .map("{:02d}".format)
    .isin(material_type)
]

In [10]:
select_mm_matType = mm[mm["Material_Type"] == 6.0]
select_mm_matType

,Material_Type,Material_Number,Material_Description,PlantID,Plant,Plant_Description,UOM,Moving_avg_price,Planned_delivery_time,Reorder_point,...,CompanyCode,CompanyDesc,Consigment_Onhand,Consigment_Blocked,MaterialConsigment,Currency,flagdeleted,createdatetime,updatedatetime,pk_plant_matnum
8563,6.0,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)",PT011,7560,SKIC-Wangsa-la,PC,1416.66,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:26:53.72,7560_7506809480106
8564,6.0,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA",PT011,7560,SKIC-Wangsa-la,PC,36975.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:13:13.247,7560_7806055050934
8565,6.0,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,PT011,7560,SKIC-Wangsa-la,PC,1700.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:30:06.927,7560_54060106035573
8566,6.0,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034",PT011,7560,SKIC-Wangsa-la,PC,6140.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:47:51.68,7560_54060140106258
8567,6.0,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028",PT011,7560,SKIC-Wangsa-la,PC,2450.00,NaN,0.0,...,750,"SIAM KRAFT INDUSTRY CO.,LTD",0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:12:51.51,7560_54060140113243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309995,6.0,9A06LEDRYMCONT001,3RF2340-1AA06-Solid-state contactor,PT131,9A11,VISY - Store Plant,EA,10195.10,14.0,1.0,...,3200,Vexcel Pack Company Limited,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:12.773,2025-11-15 04:45:06.807,9A11_9A06LEDRYMCONT001
310004,6.0,9A06LAOTHMVALV033,4551305-Electrovalve 1078/M42S CASTEL,PT131,9A11,VISY - Store Plant,EA,8050000.00,14.0,1.0,...,3200,Vexcel Pack Company Limited,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:13.133,2025-11-15 04:45:06.95,9A11_9A06LAOTHMVALV033
310005,6.0,9A06LAOTHMVALV034,4192102-Thermost expansion VALVE SEH-17,PT131,9A11,VISY - Store Plant,EA,8050000.00,14.0,1.0,...,3200,Vexcel Pack Company Limited,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:13.15,2025-11-15 04:45:06.963,9A11_9A06LAOTHMVALV034
310018,6.0,LX06320-4207,"BUSHING, BRONZE 5"" OD X 13"" L SOLID",PT115,LX12,UPPC-Philippines,IN,1729.11,21.0,14.0,...,6590,"UNITED PULP AND PAPER CO.,INC",0.0,0.0,NaN,PHP,NaN,2025-11-17 04:45:07.987,2025-11-18 04:45:06.973,LX12_LX06320-4207


In [11]:
#unduplicate values
select_mm_matType["Material_Number"].value_counts()[lambda x: x == 1]

Material_Number
LX06340-5235-1        1
LX06340-5298          1
LX06600-0057-1        1
78061428015951        1
LX06340-5246          1
                     ..
OL06SE7802052A        1
OL06SE7802053A        1
OL06SE7802054A        1
OL06SE7803003A        1
25062A62MOTORA1-49    1
Name: count, Length: 43048, dtype: int64

In [12]:
select_mm_matType["Material_Number"].duplicated().sum()

np.int64(17385)

In [13]:
#check duplicate pk
select_mm_matType["pk_plant_matnum"].value_counts()[lambda x: x > 1]

pk_plant_matnum
259A_25069A290126002    2
C611_C60611210013001    2
259A_25069A290126003    2
Name: count, dtype: int64

In [14]:
# Deduplicate select first row
select_mm_matType_no_dup = select_mm_matType.drop_duplicates(subset=["pk_plant_matnum"], keep="first")
select_mm_matType_no_dup["pk_plant_matnum"].value_counts()[lambda x: x > 1]

Series([], Name: count, dtype: int64)

In [15]:
#duplicate values
select_mm_matType_no_dup["Material_Number"].value_counts()[lambda x: x > 1]

Material_Number
78060628955964     7
75060672337117     7
78060626102000     6
78064670512951     6
78067302594982     6
                  ..
LX06373-0031       2
75066834201211     2
75066830007236     2
25068A230140023    2
75066833902133     2
Name: count, Length: 11256, dtype: int64

In [16]:
#unduplicate values
select_mm_matType_no_dup["Material_Number"].value_counts()[lambda x: x == 1]

Material_Number
LX06336-0497          1
LX06445-0004          1
LX06336-0495          1
LX06336-0304          1
LX06336-0388          1
                     ..
OL06SE7802052A        1
OL06SE7802053A        1
OL06SE7802054A        1
OL06SE7803003A        1
25062A62MOTORA1-49    1
Name: count, Length: 43051, dtype: int64

### PO_TEXT

In [17]:

po = pd.read_csv("Vw_POText.csv", encoding="utf-8")

# แสดงข้อมูล 5 แถวแรก
print(po.head())

             Material TypeText  Line                                  Text  \
0  W0WIREWWAYACCE0013       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   
1  W0WIREWWAYACCE0021       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   
2  W0WIREWWAYACCE0029       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   
3  W0WIREWWAYACCE0037       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   
4  W0WIREWWAYACCE0045       PO     4  -  POWDER COATED IN TEXTURE RAL 7032   

   CreateBy           CreateDateTime  rowid  
0         1  2023-08-28 10:37:38.503      1  
1         1  2023-08-28 10:37:38.503      2  
2         1  2023-08-28 10:37:38.503      3  
3         1  2023-08-28 10:37:38.503      4  
4         1  2023-08-28 10:37:38.503      5  


In [18]:
po.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   Material        1048575 non-null  object
 1   TypeText        1048575 non-null  object
 2   Line            1048575 non-null  int64 
 3   Text            1027025 non-null  object
 4   CreateBy        1048575 non-null  int64 
 5   CreateDateTime  1048575 non-null  object
 6   rowid           1048575 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 56.0+ MB


In [19]:
po["Material"].value_counts()[lambda x: x > 1]

Material
W0OFFIPTWPFORM0084    14
W0INVACS355_7_5KW     14
W0SAFEPSAPACCE0035    14
RE06704118912101      14
RE06704151911100      14
                      ..
7542NS01EE00200578     2
7542NS01EE00200579     2
7542NS01EE00200580     2
7542NS01EE00200384     2
W0CHLBCHEMLABXM202     2
Name: count, Length: 152770, dtype: int64

In [20]:
# รวมข้อความตาม Material
po_combine_text = (
    po.sort_values(["Material", "Line"])                # เรียงตาม Material และ Line
      .groupby("Material")                              # จัดกลุ่มตาม Material
      .agg({"Text": lambda x: " ".join(x.astype(str))}) # รวม Text โดยคั่นด้วย space
      .reset_index()                                     # ให้ Material กลับมาเป็น column
)

po_combine_text

,Material,Text
0,210STKR0000018,STK.PE ใสหลังกลาสซีน กาวธรรมดา 180mmx1000m #BW...
1,225107A400004228,CK SP CK SP33131ORANGE nan ORANGE 4228 nan nan
2,2503,"PLC, MFR PART NAME : PLC CPU, MFR PART NO. : 6..."
3,25032A000045001,"FUSE, SHORT NAME : HCR FUSE 24KV/100A, VOLTAG..."
4,25032A000045002,SENSOR S30B-2011DA#1026822 สถานที่ใช้งาน TRANS...
...,...,...
177924,W410WT140100001,"KRAFT, TYPE : LINER, MFR MODEL NO. : WT140, ..."
177925,W410WT160000001,"KRAFT, TYPE : LINER, MFR MODEL NO. : WT160, ..."
177926,W410YL140130001,KRAFT LINER SDY TL140
177927,W431FORMLW00001,แบบฟอร์มกระดาษต่อเนื่อง ใบกำกับภาษี/ใบส่งของ ก...


## map mat num betweem mat_mas & po text

In [21]:
select_mm_matType_join = select_mm_matType_no_dup.merge(
    po_combine_text,
    how="left",
    left_on="Material_Number",
    right_on="Material"
)

In [22]:
select_mm_matType_join

,Material_Type,Material_Number,Material_Description,PlantID,Plant,Plant_Description,UOM,Moving_avg_price,Planned_delivery_time,Reorder_point,...,Consigment_Onhand,Consigment_Blocked,MaterialConsigment,Currency,flagdeleted,createdatetime,updatedatetime,pk_plant_matnum,Material,Text
0,6.0,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)",PT011,7560,SKIC-Wangsa-la,PC,1416.66,NaN,0.0,...,0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:26:53.72,7560_7506809480106,7506809480106,"SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ..."
1,6.0,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA",PT011,7560,SKIC-Wangsa-la,PC,36975.00,NaN,0.0,...,0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:13:13.247,7560_7806055050934,7806055050934,"FLOWMETER, SHORT NAME : FLOWMETER, FLOW RANGE..."
2,6.0,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,PT011,7560,SKIC-Wangsa-la,PC,1700.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:30:06.927,7560_54060106035573,54060106035573,"ACID PHOSPHORIC COMERCIAL GRADE, MACHINE CODE ..."
3,6.0,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034",PT011,7560,SKIC-Wangsa-la,PC,6140.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:47:51.68,7560_54060140106258,54060140106258,"SHORT NAME : BEARING,ACC, MFR NAME : SKF, TY..."
4,6.0,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028",PT011,7560,SKIC-Wangsa-la,PC,2450.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:12:51.51,7560_54060140113243,54060140113243,"BEARING,ACC, MFR NAME : SKF, TYPE : WITHDRAW..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71684,6.0,9A06LEDRYMCONT001,3RF2340-1AA06-Solid-state contactor,PT131,9A11,VISY - Store Plant,EA,10195.10,14.0,1.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:12.773,2025-11-15 04:45:06.807,9A11_9A06LEDRYMCONT001,NaN,NaN
71685,6.0,9A06LAOTHMVALV033,4551305-Electrovalve 1078/M42S CASTEL,PT131,9A11,VISY - Store Plant,EA,8050000.00,14.0,1.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:13.133,2025-11-15 04:45:06.95,9A11_9A06LAOTHMVALV033,NaN,NaN
71686,6.0,9A06LAOTHMVALV034,4192102-Thermost expansion VALVE SEH-17,PT131,9A11,VISY - Store Plant,EA,8050000.00,14.0,1.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:13.15,2025-11-15 04:45:06.963,9A11_9A06LAOTHMVALV034,NaN,NaN
71687,6.0,LX06320-4207,"BUSHING, BRONZE 5"" OD X 13"" L SOLID",PT115,LX12,UPPC-Philippines,IN,1729.11,21.0,14.0,...,0.0,0.0,NaN,PHP,NaN,2025-11-17 04:45:07.987,2025-11-18 04:45:06.973,LX12_LX06320-4207,LX06320-4207,"Bushing, Bronze size: 5"" OD x 13"" Length , Sol..."


In [23]:
select_mm_matType_mapped = select_mm_matType_join[select_mm_matType_join["Material"].notna()]
select_mm_matType_mapped

,Material_Type,Material_Number,Material_Description,PlantID,Plant,Plant_Description,UOM,Moving_avg_price,Planned_delivery_time,Reorder_point,...,Consigment_Onhand,Consigment_Blocked,MaterialConsigment,Currency,flagdeleted,createdatetime,updatedatetime,pk_plant_matnum,Material,Text
0,6.0,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)",PT011,7560,SKIC-Wangsa-la,PC,1416.66,NaN,0.0,...,0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:26:53.72,7560_7506809480106,7506809480106,"SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ..."
1,6.0,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA",PT011,7560,SKIC-Wangsa-la,PC,36975.00,NaN,0.0,...,0.0,0.0,NaN,THB,X,2018-07-14 20:41:05.44,2024-12-11 11:13:13.247,7560_7806055050934,7806055050934,"FLOWMETER, SHORT NAME : FLOWMETER, FLOW RANGE..."
2,6.0,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,PT011,7560,SKIC-Wangsa-la,PC,1700.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:30:06.927,7560_54060106035573,54060106035573,"ACID PHOSPHORIC COMERCIAL GRADE, MACHINE CODE ..."
3,6.0,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034",PT011,7560,SKIC-Wangsa-la,PC,6140.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:47:51.68,7560_54060140106258,54060140106258,"SHORT NAME : BEARING,ACC, MFR NAME : SKF, TY..."
4,6.0,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028",PT011,7560,SKIC-Wangsa-la,PC,2450.00,NaN,0.0,...,0.0,0.0,NaN,THB,NaN,2018-07-14 20:41:05.44,2025-03-27 22:12:51.51,7560_54060140113243,54060140113243,"BEARING,ACC, MFR NAME : SKF, TYPE : WITHDRAW..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71672,6.0,78062409570000,"COUPLING,COMPLETE,DODGE,PX70,DODGE(5)",PT011,7560,SKIC-Wangsa-la,ST,9630.00,0.0,0.0,...,0.0,0.0,NaN,THB,NaN,2025-11-11 04:45:07.363,2025-11-13 04:45:08.85,7560_78062409570000,78062409570000,"COUPLING, SHORT NAME : COUPLING, MFR PART NAM..."
71681,6.0,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC",PT016,7813,TPC-Ban Pong,PC,3865.00,0.0,0.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:07.173,2025-11-15 04:45:04.583,7813_78063621820573,78063621820573,"FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."
71682,6.0,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC",PT014,7811,TPC-Ban Pong,PC,3865.00,0.0,0.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:08.417,2025-11-15 04:45:04.69,7811_78063621820573,78063621820573,"FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."
71683,6.0,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC",PT022,K011,SNP-Ban Pong,PC,3865.00,0.0,0.0,...,0.0,0.0,NaN,THB,NaN,2025-11-14 04:45:08.953,2025-11-15 04:45:05.063,K011_78063621820573,78063621820573,"FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."


In [24]:
select_mm_matType_mapped["pk_plant_matnum"].value_counts()[lambda x: x > 1]

Series([], Name: count, dtype: int64)

In [25]:
select_select_mm_matType = select_mm_matType_mapped[["pk_plant_matnum", "Material_Type", "Plant", "Material_Number", "PlantID", "Plant_Description", "UOM", "Material", "Material_Description", "Text"]]
select_select_mm_matType

,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text
0,7560_7506809480106,6.0,7560,7506809480106,PT011,SKIC-Wangsa-la,PC,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)","SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ..."
1,7560_7806055050934,6.0,7560,7806055050934,PT011,SKIC-Wangsa-la,PC,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA","FLOWMETER, SHORT NAME : FLOWMETER, FLOW RANGE..."
2,7560_54060106035573,6.0,7560,54060106035573,PT011,SKIC-Wangsa-la,PC,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,"ACID PHOSPHORIC COMERCIAL GRADE, MACHINE CODE ..."
3,7560_54060140106258,6.0,7560,54060140106258,PT011,SKIC-Wangsa-la,PC,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034","SHORT NAME : BEARING,ACC, MFR NAME : SKF, TY..."
4,7560_54060140113243,6.0,7560,54060140113243,PT011,SKIC-Wangsa-la,PC,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028","BEARING,ACC, MFR NAME : SKF, TYPE : WITHDRAW..."
...,...,...,...,...,...,...,...,...,...,...
71672,7560_78062409570000,6.0,7560,78062409570000,PT011,SKIC-Wangsa-la,ST,78062409570000,"COUPLING,COMPLETE,DODGE,PX70,DODGE(5)","COUPLING, SHORT NAME : COUPLING, MFR PART NAM..."
71681,7813_78063621820573,6.0,7813,78063621820573,PT016,TPC-Ban Pong,PC,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC","FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."
71682,7811_78063621820573,6.0,7811,78063621820573,PT014,TPC-Ban Pong,PC,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC","FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."
71683,K011_78063621820573,6.0,K011,78063621820573,PT022,SNP-Ban Pong,PC,78063621820573,"FILTER,OIL,0250 DN010 BN3HC,HYDAC","FILTER,OIL, MFR MODEL NO. : 0250 DN010 BN3HC,..."


In [26]:
print(select_select_mm_matType[["Material_Description","UOM", "Text"]]
      .head()
      .to_string())

                     Material_Description UOM                                                                                                                                                                                                                                                                                                                                                                                                                                                                 Text
0          SPROCKET,CHAIN,49 T ,1/2 "(ZN)  PC                                                                                                          SPROCKET,CHAIN, SHORT NAME  : SPROCKET,CHAIN, NO. OF STRANDS  : 1, NO. OF TEETH  : 49, FOR CHAIN NO.  : ?, PITCH  : 1/2 ", MATERIAL  : CARBON STEEL, APPLICATION  : SILENT CHAIN STOKER PB 8, REMARK  : SPROCKET DRIVEN MAT. MEDIUM CARBON STEEL TOOTH HARD SURFACE NO. OF TEETH 49, PITCH 0.500" AS PER DWG.NO. 106 309 B 001 FOR SILENT CHAIN STOKER PB 8
1 

In [27]:
select_select_mm_matType["UOM"].value_counts().count()

np.int64(40)

In [28]:
select_select_mm_matType.to_csv("select_select_mm_matType_06.csv", index=False, encoding="utf-8-sig")

## 0. Combine field

### 0.1 for raw_combine_for_embedding (without uom)

In [29]:
select_select_mm_matType["raw_combine_for_embedding"] = "short_description: " + select_select_mm_matType["Material_Description"] + " long_description: " + select_select_mm_matType["Text"]
select_select_mm_matType.head(1)

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_56111/721366568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["raw_combine_for_embedding"] = "short_description: " + select_select_mm_matType["Material_Description"] + " long_description: " + select_select_mm_matType["Text"]


,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text,raw_combine_for_embedding
0,7560_7506809480106,6.0,7560,7506809480106,PT011,SKIC-Wangsa-la,PC,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)","SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ...","short_description: SPROCKET,CHAIN,49 T ,1/2 ""(..."


### 0.2 for raw_combine_for_keyword (add uom)

In [30]:
select_select_mm_matType["raw_combine_for_keyword"] = select_select_mm_matType["Material_Description"] + " " + select_select_mm_matType["Text"] + " " + select_select_mm_matType["UOM"]
select_select_mm_matType.head(1)

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_56111/1177480277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["raw_combine_for_keyword"] = select_select_mm_matType["Material_Description"] + " " + select_select_mm_matType["Text"] + " " + select_select_mm_matType["UOM"]


,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text,raw_combine_for_embedding,raw_combine_for_keyword
0,7560_7506809480106,6.0,7560,7506809480106,PT011,SKIC-Wangsa-la,PC,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)","SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ...","short_description: SPROCKET,CHAIN,49 T ,1/2 ""(...","SPROCKET,CHAIN,49 T ,1/2 ""(ZN) SPROCKET,CHAIN,..."


## 1. Data Cleaning & Normalization
สรุป: ทำให้ข้อมูลสะอาด สม่ำเสมอ พร้อมให้ embedding และ BM25 ใช้ได้ดี
- Uppercase ทั้งหมด
- Normalize whitespace
- เก็บ character สำคัญของ part code (- / _ . ( ))
- Remove noise tokens (EA, PCS, ITEM, DESC)
- รวม material_description + PO_text → context

In [31]:
def normalize_whitespace(text: str) -> str:
    # แทน \n \t ด้วย space และลด space ซ้อน
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def remove_duplicate_commas_periods(text: str) -> str:
    # ลด ,, → , และ .. → .
    text = re.sub(r",{2,}", ",", text)
    text = re.sub(r"\.{2,}", ".", text)
    return text

def lowercase_text(text: str) -> str:
    return text.lower()


# --------- Main Cleaning Function ---------
def cleaning_features(text: str) -> str:
    if pd.isna(text):
        return ""

    # 1. normalize whitespace
    text = normalize_whitespace(text)

    # 2. remove duplicate punctuation
    text = remove_duplicate_commas_periods(text)

    # 3. keep punctuation that conveys meaning → (ไม่มีการลบใด ๆ)

    # 4. lowercase for embedding stability
    # text = lowercase_text(text)

    # 5. Normalize dash/underscore variants
    text = normalize_dashes(text)

    return text.strip()

def normalize_dashes(text):
    return text.replace("–", "-").replace("—", "-")

def extract_partcodes(text):
    part_pattern = r"(?=[A-Z0-9]*[A-Z])[A-Z0-9]+(?:[-_/\.][A-Z0-9]+)+"
    parts = re.findall(part_pattern, text)

    # unique & preserve order
    return list(dict.fromkeys(parts))

# --------- Build Embedding Text ---------
def build_raw_combine_for_embedding(row):
    sd = cleaning_features(row["Material_Description"]) if pd.notna(row["Material_Description"]) else ""
    ld = cleaning_features(row["Text"]) if pd.notna(row["Text"]) else ""

    combined = f"short_description: {sd} long_description: {ld}"

    # extract part-code to reinforce semantic
    parts = extract_partcodes(combined)
    if parts:
        combined += " part_codes: " + ", ".join(parts)

    # lowercase everything for embedding stability
    combined = lowercase_text(combined)

    return combined.strip()

### 1.1 cleansing for sementic search (embed)
- 1. normalize whitespace
- 2. remove duplicate punctuation
- 3. Normalize dash/underscore variants
- 4. extract part-code to reinforce semantic
- 5. lowercase everything for embedding stability

In [32]:
select_select_mm_matType["clean_combine_for_embedding"] = (
    select_select_mm_matType.apply(build_raw_combine_for_embedding, axis=1)
)

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_56111/420435984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["clean_combine_for_embedding"] = (


In [33]:
with pd.option_context('display.max_colwidth', None):
    print(
        select_select_mm_matType["clean_combine_for_embedding"]
            .head(5)
            .to_string()
    )

0                                                                                                                                                                                                                                                                               short_description: sprocket,chain,49 t ,1/2 "(zn) long_description: sprocket,chain, short name : sprocket,chain, no. of strands : 1, no. of teeth : 49, for chain no. : ?, pitch : 1/2 ", material : carbon steel, application : silent chain stoker pb 8, remark : sprocket driven mat. medium carbon steel tooth hard surface no. of teeth 49, pitch 0.500" as per dwg.no. 106 309 b 001 for silent chain stoker pb 8 part_codes: dwg.no
1                                                                                                                                                                                                                                                                                                            

### 1.2 cleansing for keyword search

In [34]:
with pd.option_context('display.max_colwidth', None):
    print(
        select_select_mm_matType["raw_combine_for_keyword"]
            .head(20)
            .to_string()
    )

0                                                                                                                                                                   SPROCKET,CHAIN,49 T ,1/2 "(ZN) SPROCKET,CHAIN, SHORT NAME  : SPROCKET,CHAIN, NO. OF STRANDS  : 1, NO. OF TEETH  : 49, FOR CHAIN NO.  : ?, PITCH  : 1/2 ", MATERIAL  : CARBON STEEL, APPLICATION  : SILENT CHAIN STOKER PB 8, REMARK  : SPROCKET DRIVEN MAT. MEDIUM CARBON STEEL TOOTH HARD SURFACE NO. OF TEETH 49, PITCH 0.500" AS PER DWG.NO. 106 309 B 001 FOR SILENT CHAIN STOKER PB 8 PC
1                                                                                                                                                                                                                                                                                                                                                                   FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA FLOWMETER, SHORT NAME  : FLOWMETER, FLOW RANGE  : 300L/MIN, MFR

In [35]:
# ยังติด no. ใน keyword_list

In [36]:
# Function before extracting identifier_keyword
def clean_quotes(text: str) -> str:
    """
    - remove "word" ที่ไม่มีเลขด้านใน
    - remove stray quotes เช่น "zn → zn
    - แปลง inch: 3/4" → 3/4 inch
    - (สุดท้ายค่อย normalize spacing)
    """

    # ---------------------------------------------------------
    # A) remove quoted words without digits → remove entirely
    # "SKF" → space
    # "MAHLE" → space
    # "abc-def" → space
    # ---------------------------------------------------------
    def remove_non_numeric_quotes(m):
        inside = m.group(1)
        if not re.search(r'\d', inside):
            return " "     # ไม่มีเลข → remove
        return m.group(0)  # มีเลข → keep

    text = re.sub(r'"([^"]*)"', remove_non_numeric_quotes, text)

    # ---------------------------------------------------------
    # B) remove stray quotes (quote เดี่ยว ๆ ที่เหลือ)
    # เช่น "zn → zn,  zn" → zn
    # ---------------------------------------------------------
    text = re.sub(r'(^| )"(\w+)', r'\1\2', text)
    text = re.sub(r'(\w+)"( |$)', r'\1\2', text)

    # ---------------------------------------------------------
    # C) convert inch (only numeric + double-quote)
    # ทำหลังจาก clean quotes แล้ว
    # ---------------------------------------------------------
    text = re.sub(r'(\d+/\d+|\d+(?:\.\d+)?)"', r'\1 inch', text)
    text = re.sub(r'(\d+/\d+|\d+(?:\.\d+)?)\s*"(\s|$)', r'\1 inch ', text)

    # ---------------------------------------------------------
    # D) normalize spaces
    # ---------------------------------------------------------
    text = re.sub(r'\s+', ' ', text).strip()

    return text

special_chars = ["?"]

def clean_special_chars(text: str, special_chars: list) -> str:
    """
    Remove / replace special characters defined in special_chars list.
    Replace with space to preserve token boundaries.
    """
    if not text:
        return text

    # escape chars for regex safety (เช่น ?, *, ., etc.)
    escaped = [re.escape(c) for c in special_chars]
    pattern = f"[{''.join(escaped)}]"

    # replace with space
    text = re.sub(pattern, " ", text)

    # collapse multiple spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text


def normalize_spacing(text):
    """
    - ใส่ space ก่อน–หลัง comma
    - compress multiple spaces
    """

    # Add space around commas
    text = re.sub(r'\s*,\s*', ' , ', text)

    # collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def normalize_uom(text: str) -> str:

    # Convert number + optional space + " → number inch
    # Examples:
    #   1/2"  → 1/2 inch
    #   1/2 " → 1/2 inch
    #   10"   → 10 inch
    #   10 "  → 10 inch
    text = re.sub(r'\b(\d+(?:/\d+)?)(\s*)"(?![a-zA-Z])', r'\1 inch', text)

    # --- 2. millimeter → mm ---
    text = re.sub(r'\bmil{0,1}imet(er|re|ers|res)\b', 'mm', text)

    # --- 3. kg variations → kg ---
    text = re.sub(r'\bkg\.?\b', 'kg', text)
    text = re.sub(r'\bkgs?\.?\b', 'kg', text)
    text = re.sub(r'\bkilogram(s)?\b', 'kg', text)

    # --- 4. bars → bar ---
    text = re.sub(r'\bbars\b', 'bar', text)

    # --- 5. centimeter → cm ---
    text = re.sub(r'\bcentimet(er|re|ers|res)\b', 'cm', text)
    text = re.sub(r'\bcentimeter(s)?\b', 'cm', text)

    # --- 6. meter → m ---  
    # (ไม่แตะกรณี M560475D1 เพราะต้องเป็น word boundary)
    text = re.sub(r'\bmet(er|re|ers|res)\b', 'm', text)
    text = re.sub(r'\bmeters\b', 'm', text)

    # --- 7. liter → L ---
    text = re.sub(r'\blitre(s)?\b', 'L', text)
    text = re.sub(r'\bliter(s)?\b', 'L', text)

    # --- 8. ton / tonne ---
    text = re.sub(r'\btonne(s)?\b', 'ton', text)
    text = re.sub(r'\btons\b', 'ton', text)

    return text

# ============================================================
# 2) Preprocess text (normalize punctuation / spacing / dash)
# ============================================================
def normalize_text(t: str) -> str:
    t = t.lower()

    # normalize dash variations
    t = t.replace("–", "-").replace("—", "-").replace("_", "-")

    # remove duplicate punctuations (,,, ---> ,)
    # reduce duplicate punctuation (safe)
    t = re.sub(r'([!?,.;]{2,})', lambda m: m.group()[0], t)

    # # remove unwanted punctuations: * and :
    t = t.replace("*", " ")
    t = t.replace(":", " ")

    # # normalize spaces
    t = re.sub(r"\s+", " ", t).strip()

    return t

def normalize_temperature(text):
    # -------- Celsius --------
    # ครอบคลุม:
    # DEGREE CELSIUS, DEG CELSISU, DEG C., 
    #  C, DEG.CELSIUS,
    # DEGREECELSIUS, DEGREE OF CELSIUS, DEGREE OF C, DEG C.
    celsius_patterns = [
        r'\b(\d+)\s*(deg\.?|degree\.?|degrees\.?)\s*(of\s*)?(celsius|celcius|celsisu|cels|cel|c)\b',
        r'\b(\d+)\s*(°)\s*c\b',
        r'\b(\d+)\s*c\b'
    ]

    for p in celsius_patterns:
        text = re.sub(p, r'\1°C', text, flags=re.I)

    # -------- Fahrenheit --------
    fahrenheit_patterns = [
        r'\b(\d+)\s*(deg\.?|degree\.?|degrees\.?)\s*(of\s*)?(fahrenheit|farenheit|farenhiet|f)\b',
        r'\b(\d+)\s*(°)\s*f\b',
        r'\b(\d+)\s*f\b'
    ]

    for p in fahrenheit_patterns:
        text = re.sub(p, r'\1°F', text, flags=re.I)

    return text

def normalize_technical_terms(text: str) -> str:
    # ------------------------------------------------------------
    # STEP 1) handle dia. followed by number → insert space only
    # ------------------------------------------------------------
    text = re.sub(r'\bdia\.(?=\d)', r'dia. ', text, flags=re.I)

    # ------------------------------------------------------------
    # STEP 2) normalize exact tokens only → dia.
    #   - diameter
    #   - diam.
    #   - dia
    #   - dia.  (เฉพาะกรณีเป็น token เดี่ยว ๆ)
    #   - ø φ Ø Φ
    # ------------------------------------------------------------
    # normalize diameter
    text = re.sub(r'\bdiameter\b', 'dia.', text, flags=re.I)

    # normalize diam.
    text = re.sub(r'\bdiam\.\b', 'dia.', text, flags=re.I)

    # normalize symbol forms
    text = re.sub(r'[ØøφΦ⌀]', 'dia.', text)

    # normalize dia. (แต่ต้องเป็น token เดี่ยว ไม่ติดตัวเลข)
    text = re.sub(r'\bdia\.(?!\d)', 'dia.', text, flags=re.I)

    # ------------------------------------------------------------
    # STEP 3) collapse spaces
    # ------------------------------------------------------------
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [37]:
# Temp for extract_identifier_candidates
# ============================================================
# 1) Utility regex
# ============================================================

# แนวๆ CODE/PART NUMBER:
code_patterns = [
    r"[A-Z]{1,4}\d{1,6}[A-Z0-9\-]*",    # ABC1234 / E70340M-K059521
    r"\d{2,4}[A-Z]{1,4}\d{2,6}",        # 512ABC1234
    r"[A-Z]{1,4}\d{2,4}",               # TLFH-4CA (ก่อน normalize)
    r"\d{2,4}[A-Z]{1,4}",               
    # r"\bAH[- ]?\d{3,5}\b",              # AH24034
]

code_regex = re.compile("|".join(code_patterns), re.IGNORECASE)
# ============================================================
# 4) Extract CODE patterns for exact-match boosting
# ============================================================
def extract_codes(t: str):
    return list(set(code_regex.findall(t)))

In [38]:
# Extract identifier keyword
# 1. extract_identifier_candidates

IDENTIFIER_PATTERNS = [

    # 1) Classic material / part code
    r"\b[A-Z]{1,4}[- ]?\d{2,6}[A-Z0-9]*\b",

    # 2) Complex code with separators
    r"\b[A-Z0-9]{2,}[-/][A-Z0-9-/]{2,}\b",

    # 3) Drawing / model no.
    r"\b[A-Z]{1,3}\d{4,}[A-Z0-9/.-]*\b",

    # 4) Metric thread / fastener
    r"\bM\d{2,3}\s*[xX]\s*L?\d{2,4}\b",

    # 5) Dimension compound
    r"\b\d{2,4}\s*[xX]\s*\d{2,5}\s*(mm|cm|m)?\b",

    # 6) Pipe / standard
    r"\bDN\d{2,4}\b",
]

IDENTIFIER_REGEX = re.compile("|".join(IDENTIFIER_PATTERNS), re.IGNORECASE)


def extract_identifier_candidates(text: str) -> list[str]:
    if not isinstance(text, str):
        return []
    return list(set(m.group(0) for m in IDENTIFIER_REGEX.finditer(text)))


# 2. normalize_identifier
def normalize_identifier(code: str) -> str:
    c = code.strip().upper()

    # 1) normalize spaces around separators
    c = re.sub(r"\s*-\s*", "-", c)
    c = re.sub(r"\s*/\s*", "/", c)
    c = re.sub(r"\s*[xX]\s*", "x", c)

    # 2) AH 24034 → AH-24034
    c = re.sub(r"\b([A-Z]{1,4})\s+(\d{2,6})\b", r"\1-\2", c)

    # 3) normalize unit casing
    c = re.sub(r"\bMM\b", "MM", c)

    return c

# 3. filter_identifier
def is_valid_identifier(code: str) -> bool:

    # too short
    if len(code) < 4:
        return False

    # pure number
    if code.isdigit():
        return False

    # single dimension like 10MM
    if re.fullmatch(r"\d{1,4}(MM|CM|M)?", code):
        return False

    # must contain at least one digit + one letter
    if not (re.search(r"[A-Z]", code) and re.search(r"\d", code)):
        return False

    return True

def extract_identifier_keywords(text: str) -> list[str]:
    raw_candidates = extract_identifier_candidates(text)

    normalized = [normalize_identifier(c) for c in raw_candidates]

    filtered = [c for c in normalized if is_valid_identifier(c)]

    return sorted(set(filtered))



In [39]:
def remove_no(text):
    """
    remove only stopword: "no."
    """
    tokens = text.split()
    tokens = [t for t in tokens if t.lower() not in ["no."]]
    return " ".join(tokens)

# ============================================================
# 5) Remove stopwords for BM25 keyword clarity
# ============================================================
# def remove_stopwords(t: str) -> str:
#     tokens = t.split()
#     tokens = [tok for tok in tokens if tok not in stopwords]
#     return " ".join(tokens)
stopwords_list = ["short",
    "name",
    "long",
    "description",
    "remark",
    "application",
    "equipment",
    "machine",
    "mfr",
    "maker",
    "model",
    "type",
    "serial",
    "supplier",
    "shop",
    "drawing",
    "dwg.",
    "dwg",
    "no.",
    "number",
    "offer",
    "item",
    "position",
    "use for",
    "used for"
    "used",
    "?",
    "from",
    "part",
    "partno",
    "code",
    "unit",
    "qty.",
    "qty",
    "size",
    "pressure",
    "range"
]

stopwords = set(stopwords_list)

def remove_stopwords(t: str) -> str:
    tokens = t.split()
    cleaned = []
    for tok in tokens:
        tok_clean = tok.strip(",:;")  # ลบ punctuation รอบ ๆ
        if tok_clean and tok_clean not in stopwords:  # <--- ป้องกัน "" 
            cleaned.append(tok_clean)
    return " ".join(cleaned)

# ============================================================
# 6) Create keyword_list field
# ============================================================
def build_keyword_list(clean_text: str, codes: list):
    tokens = clean_text.split()
    return list(sorted(set(tokens + codes)))

In [40]:
# ============================================================
# Main logic for keyword pipeline
# ============================================================
def clean_keyword_pipeline(text: str):
    if not isinstance(text, str):
        return "", [], [], []

    t = text

    # 1) special " cleaner (change " to inch / garbage / remove)
    t = clean_quotes(t)

    # 2) clean special chars
    t = clean_special_chars(t, special_chars)

    # 3) Add space around comma
    t = normalize_spacing(t)

    # # 4) normalize
    t = normalize_text(t)

    # # 5) normalize UOM
    t = normalize_uom(t)

    # # 6) normalize degree celsius and Fahrenheit
    t = normalize_temperature(t)

    # # 7) normalize technical term
    t = normalize_technical_terms(t)

    # # 8) extract codes
    old_codes = extract_codes(t)
    codes = extract_identifier_keywords(t)

    # # 9) remove no.
    t = remove_no(t)

    # 10) remove stopwords
    t_no_stop = remove_stopwords(t)

    # 5) build keyword list
    keyword_list = build_keyword_list(t_no_stop, codes)
    # codes = ""
    # keyword_list = ""

    return t, codes, old_codes, keyword_list



select_select_mm_matType["clean_combine_for_keyword"], select_select_mm_matType["keyword_codes"], select_select_mm_matType["keyword_old_codes"], select_select_mm_matType["keyword_list"] = zip(
    *select_select_mm_matType["raw_combine_for_keyword"].map(clean_keyword_pipeline)
)

select_select_mm_matType.head()

/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_56111/2088084179.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["clean_combine_for_keyword"], select_select_mm_matType["keyword_codes"], select_select_mm_matType["keyword_old_codes"], select_select_mm_matType["keyword_list"] = zip(
/var/folders/d0/9zz00l_j59ggjlmw8r4v0pn00000gn/T/ipykernel_56111/2088084179.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_select_mm_matType["clean_combine_for_keyword"], select

,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text,raw_combine_for_embedding,raw_combine_for_keyword,clean_combine_for_embedding,clean_combine_for_keyword,keyword_codes,keyword_old_codes,keyword_list
0,7560_7506809480106,6.0,7560,7506809480106,PT011,SKIC-Wangsa-la,PC,7506809480106,"SPROCKET,CHAIN,49 T ,1/2 ""(ZN)","SPROCKET,CHAIN, SHORT NAME : SPROCKET,CHAIN, ...","short_description: SPROCKET,CHAIN,49 T ,1/2 ""(...","SPROCKET,CHAIN,49 T ,1/2 ""(ZN) SPROCKET,CHAIN,...","short_description: sprocket,chain,49 t ,1/2 ""(...","sprocket , chain , 49 t , 1/2 inch(zn) sprocke...",[B-001],[],"[0.500, 001, 1, 1/2, 106, 309, 49, 8, B-001, a..."
1,7560_7806055050934,6.0,7560,7806055050934,PT011,SKIC-Wangsa-la,PC,7806055050934,"FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA","FLOWMETER, SHORT NAME : FLOWMETER, FLOW RANGE...","short_description: FLOWMETER,300L/MIN,TLFH-4CA...","FLOWMETER,300L/MIN,TLFH-4CA,KYTOLA FLOWMETER, ...","short_description: flowmeter,300l/min,tlfh-4ca...","flowmeter , 300l/min , tlfh-4ca , kytola flowm...","[300L/MIN, TLFH-4CA]",[300l],"[300L/MIN, 300l/min, TLFH-4CA, flow, flowmeter..."
2,7560_54060106035573,6.0,7560,54060106035573,PT011,SKIC-Wangsa-la,PC,54060106035573,ACID PHOSPHORIC COMERCIAL GRADE,"ACID PHOSPHORIC COMERCIAL GRADE, MACHINE CODE ...",short_description: ACID PHOSPHORIC COMERCIAL G...,ACID PHOSPHORIC COMERCIAL GRADE ACID PHOSPHORI...,short_description: acid phosphoric comercial g...,acid phosphoric comercial grade acid phosphori...,"[E70340M, E70540M, K059001, K059088, K059249, ...","[e70540m-l014001, e70540m-l014004, e70340m-k05...","[(กรดบัดกรี), 35, 512, E70340M, E70540M, K0590..."
3,7560_54060140106258,6.0,7560,54060140106258,PT011,SKIC-Wangsa-la,PC,54060140106258,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034","SHORT NAME : BEARING,ACC, MFR NAME : SKF, TY...","short_description: BEARING,ACC,WITHDRAWAL SLEE...","BEARING,ACC,WITHDRAWAL SLEEVE,AH 24034 SHORT N...","short_description: bearing,acc,withdrawal slee...","bearing , acc , withdrawal sleeve , ah 24034 s...","[AH-24034, M-180]",[],"[180, 24034, 3, AH-24034, M-180, acc, adapter,..."
4,7560_54060140113243,6.0,7560,54060140113243,PT011,SKIC-Wangsa-la,PC,54060140113243,"BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028","BEARING,ACC, MFR NAME : SKF, TYPE : WITHDRAW...","short_description: BEARING,ACC,WITHDRAWAL SLEE...","BEARING,ACC,WITHDRAWAL SLEEVE,AH 24028 BEARING...","short_description: bearing,acc,withdrawal slee...","bearing , acc , withdrawal sleeve , ah 24028 b...",[AH-24028],[],"[2, 24028, AH-24028, acc, adapter, ah, bearing..."


In [41]:
with pd.option_context('display.max_colwidth', None):
    print(
        select_select_mm_matType[["pk_plant_matnum","raw_combine_for_keyword","clean_combine_for_keyword","keyword_codes","keyword_old_codes","keyword_list"]]
            .head(30)
            .to_string()
    )

        pk_plant_matnum                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          raw_combine_for_keyword                                                                                                                                                                                                                                                                                                                                                                                                

In [42]:
select_select_mm_matType[
    ["raw_combine_for_keyword", "clean_combine_for_keyword", "keyword_codes", "keyword_list"]
].head(30).to_csv(
    "check_30_new1.csv",
    index=False,
    encoding="utf-8-sig"
)

In [43]:
edges = pd.read_csv("edges.csv")
sample20 = edges.head(20)
sample20.head()

,MODEL_BRAND,Key_mat_1,Plant_1,Mat_1,Key_mat_2,Plant_2,Mat_2
0,3310 ATN9_SKF,7560_75060602310346,7560,75060602310346,7511_75060610300414,7511,75060610300414
1,3311 ANR_SKF,7560_75060602411346,7560,75060602411346,7511_75060610310502,7511,75060610310502
2,7306 BECBM_SKF,7812_78060606306581,7812,78060606306581,7511_75060610805312,7511,75060610805312
3,NU 308 ECP/C3_SKF,7812_78060612308552,7812,78060612308552,7511_75060612500369,7511,75060612500369
4,C 2234 K/C4_SKF,7560_75060612234350,7560,75060612234350,7511_75060612556307,7511,75060612556307


In [44]:
# สร้าง set ของ key เพื่อให้เช็กได้เร็ว
valid_keys = set(select_select_mm_matType["pk_plant_matnum"])

# สร้าง mask ว่า record ไหนผ่านเงื่อนไข (ต้องมีทั้งคู่)
mask = (
    sample20["Key_mat_1"].isin(valid_keys) &
    sample20["Key_mat_2"].isin(valid_keys)
)

# นับจำนวน record ที่ผ่าน
count_valid_records = mask.sum()

print("จำนวน record ที่มี Key_mat_1 และ Key_mat_2 ครบทั้งคู่:", count_valid_records)


จำนวน record ที่มี Key_mat_1 และ Key_mat_2 ครบทั้งคู่: 20


In [45]:
valid_records = sample20[mask]
valid_records


,MODEL_BRAND,Key_mat_1,Plant_1,Mat_1,Key_mat_2,Plant_2,Mat_2
0,3310 ATN9_SKF,7560_75060602310346,7560,75060602310346,7511_75060610300414,7511,75060610300414
1,3311 ANR_SKF,7560_75060602411346,7560,75060602411346,7511_75060610310502,7511,75060610310502
2,7306 BECBM_SKF,7812_78060606306581,7812,78060606306581,7511_75060610805312,7511,75060610805312
3,NU 308 ECP/C3_SKF,7812_78060612308552,7812,78060612308552,7511_75060612500369,7511,75060612500369
4,C 2234 K/C4_SKF,7560_75060612234350,7560,75060612234350,7511_75060612556307,7511,75060612556307
5,NJ 308 ECP_SKF,7812_78060610608574,7812,78060610608574,7511_75060614010795,7511,75060614010795
6,NJ 210 ECJ_SKF,7812_78060610510553,7812,78060610510553,7511_75060615877000,7511,75060615877000
7,NU 222 ECJ_SKF,PO11_PO06131331609,PO11,PO06131331609,7511_75060615930000,7511,75060615930000
8,NU 318 ECJ_SKF,7560_75060612318999,7560,75060612318999,7511_75060615934000,7511,75060615934000
9,NU 320 ECJ_SKF,7560_75060612320999,7560,75060612320999,7511_75060615936000,7511,75060615936000


In [46]:
# ---------------------------
# 1) เตรียม key 40 ค่า
# ---------------------------
keys_40 = pd.concat([
    sample20["Key_mat_1"],
    sample20["Key_mat_2"]
]).dropna().unique()

# ---------------------------
# 2) ดึง 40 record จาก table หลัก
# ---------------------------
df_40 = select_select_mm_matType[
    select_select_mm_matType["pk_plant_matnum"].isin(keys_40)
]

print("จำนวน record จาก key_mat:", len(df_40))

# ---------------------------
# 3) ตัด record เหล่านี้ออกจาก pool
# ---------------------------
remaining_pool = select_select_mm_matType[
    ~select_select_mm_matType["pk_plant_matnum"].isin(df_40["pk_plant_matnum"])
]

# ---------------------------
# 4) สุ่มเพิ่ม 60 record (lock seed)
# ---------------------------
df_60 = remaining_pool.sample(
    n=60,
    random_state=42   # 🔒 lock seed
)

# ---------------------------
# 5) รวมเป็น sample size 100
# ---------------------------
sample_100 = pd.concat([df_40, df_60], ignore_index=True)

print("จำนวน sample ทั้งหมด:", len(sample_100))


จำนวน record จาก key_mat: 34
จำนวน sample ทั้งหมด: 94


In [47]:
sample_100["pk_plant_matnum"].is_unique

True

In [48]:
# # สมมติ df คือ DataFrame ที่มี mat_desc และ po_text
# # df = ...

# # -----------------------------
# # 1) ฟังก์ชันทำความสะอาดข้อความ
# # -----------------------------
# def clean_text(text):

#     if pd.isna(text):
#         return ""

#     # 1. uppercase
#     text = text.upper()

#     # 2. ลบ character ที่ไม่ใช่ตัวอักษร/ตัวเลข และ ไม่ใช่ - / _ . ( )
#     # อนุญาต: A-Z, 0-9, space, '-', '/', '_', '.', '(', ')'
#     text = re.sub(r"[^A-Z0-9\-/_.() ]+", " ", text)

#     # 3. Normalize whitespace (ลดหลาย space ให้เหลือ 1 space)
#     text = re.sub(r"\s+", " ", text).strip()

#     # 4. Remove noise tokens แบบแยกคำ
#     noise = ["EA", "PCS", "ITEM", "DESC"]
#     tokens = [t for t in text.split() if t not in noise]
#     text = " ".join(tokens)

#     return text


In [49]:
# # -----------------------------
# # 2) นำฟังก์ชันไปใช้กับ mat_desc และ po_text
# # -----------------------------
# df["mat_desc_clean"] = df["mat_desc"].apply(clean_text)
# df["po_text_clean"] = df["po_text"].apply(clean_text)

# # -----------------------------
# # 3) รวม description + PO text เป็น context
# # -----------------------------
# df["context"] = (df["mat_desc_clean"] + " " + df["po_text_clean"]).str.strip()

In [50]:
# %pip install psycopg2-binary pgvector

In [51]:
# %pip install sentence-transformers

## SQL Schema for spare_part_sample Table

Schema สำหรับ Hybrid Search (Semantic + Keyword + Trigram)

In [52]:
sql_create_table = """
-- ============================================================
-- Create spare_part_sample table for Hybrid Search
-- ============================================================

-- 1) Enable required extensions
CREATE EXTENSION IF NOT EXISTS vector;
CREATE EXTENSION IF NOT EXISTS pg_trgm;

-- 2) Create main table
CREATE TABLE IF NOT EXISTS spare_part_sample (
    -- Primary Key
    pk_plant_matnum TEXT PRIMARY KEY,
    
    -- Basic Info
    material_type TEXT,
    plant TEXT,
    material_number TEXT,
    plant_id TEXT,
    plant_description TEXT,
    uom TEXT,
    
    -- Description Fields (Raw)
    material_description TEXT,
    po_text TEXT,
    
    -- ============================================================
    -- Fields for SEMANTIC SEARCH (Embedding)
    -- ============================================================
    raw_combine_for_embedding TEXT,        -- original combined text
    clean_combine_for_embedding TEXT,      -- cleaned & normalized
    embedding vector(1024),                -- bge-m3 embeddings (1024-dim)
    
    -- ============================================================
    -- Fields for KEYWORD SEARCH (BM25)
    -- ============================================================
    raw_combine_for_keyword TEXT,          -- original: desc + po_text + uom
    clean_combine_for_keyword TEXT,        -- cleaned text for BM25
    tsv tsvector,                          -- tokenized for full-text search
    
    -- Extracted Identifiers (for exact match boosting)
    keyword_codes TEXT[],                  -- extracted part codes/identifiers
    keyword_old_codes TEXT[],              -- fallback codes from old pattern
    keyword_list TEXT[],                   -- all keywords + codes combined
    
    -- ============================================================
    -- Metadata
    -- ============================================================
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- ============================================================
-- 3) Create Indexes for Performance
-- ============================================================

-- A) SEMANTIC SEARCH: Vector index (IVFFlat for cosine similarity)
DO $$
BEGIN
    IF NOT EXISTS (
        SELECT 1 FROM pg_indexes 
        WHERE tablename='spare_part_sample' 
        AND indexname='idx_spare_part_embedding'
    ) THEN
        CREATE INDEX idx_spare_part_embedding 
        ON spare_part_sample 
        USING ivfflat (embedding vector_cosine_ops)
        WITH (lists = 100);  -- adjust based on dataset size
    END IF;
END
$$;

-- B) KEYWORD SEARCH: Full-text search index (GIN for tsvector)
CREATE INDEX IF NOT EXISTS idx_spare_part_tsv 
ON spare_part_sample 
USING gin(tsv);

-- C) TRIGRAM SEARCH: Fuzzy matching on pk_plant_matnum
CREATE INDEX IF NOT EXISTS idx_spare_part_pk_trgm 
ON spare_part_sample 
USING gin (pk_plant_matnum gin_trgm_ops);

-- D) ARRAY SEARCH: GIN index on keyword arrays for exact code matching
CREATE INDEX IF NOT EXISTS idx_spare_part_keyword_codes 
ON spare_part_sample 
USING gin (keyword_codes);

CREATE INDEX IF NOT EXISTS idx_spare_part_keyword_list 
ON spare_part_sample 
USING gin (keyword_list);

-- E) BASIC INDEXES: For filtering and lookups
CREATE INDEX IF NOT EXISTS idx_spare_part_material_type 
ON spare_part_sample (material_type);

CREATE INDEX IF NOT EXISTS idx_spare_part_uom 
ON spare_part_sample (uom);

CREATE INDEX IF NOT EXISTS idx_spare_part_plant 
ON spare_part_sample (plant);

-- ============================================================
-- 4) Create Function to Auto-Update tsv on Insert/Update
-- ============================================================
CREATE OR REPLACE FUNCTION spare_part_sample_tsv_trigger() 
RETURNS TRIGGER AS $$
BEGIN
    NEW.tsv := to_tsvector('simple', COALESCE(NEW.clean_combine_for_keyword, ''));
    NEW.updated_at := CURRENT_TIMESTAMP;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Attach trigger
DROP TRIGGER IF EXISTS tsv_update_trigger ON spare_part_sample;
CREATE TRIGGER tsv_update_trigger 
BEFORE INSERT OR UPDATE ON spare_part_sample
FOR EACH ROW 
EXECUTE FUNCTION spare_part_sample_tsv_trigger();

-- ============================================================
-- 5) Helpful Utility Queries
-- ============================================================

-- Count total records
-- SELECT COUNT(*) FROM spare_part_sample;

-- Check if embeddings are populated
-- SELECT COUNT(*) FROM spare_part_sample WHERE embedding IS NOT NULL;

-- Sample data preview
-- SELECT pk_plant_matnum, material_description, uom, 
--        array_length(keyword_codes, 1) as num_codes
-- FROM spare_part_sample LIMIT 10;

"""


### Example: Hybrid Search Query with Field Weights

Query ที่รวม 3 วิธีการค้นหา + weighted scoring

In [53]:
sql_hybrid_search_example = """
-- ============================================================
-- Hybrid Search Query with Weighted Scoring
-- ============================================================

WITH 
-- 1) SEMANTIC SEARCH: Find top K by cosine similarity
semantic_search AS (
    SELECT 
        pk_plant_matnum,
        material_description,
        uom,
        (1 - (embedding <=> %s::vector)) AS semantic_score,  -- convert distance to similarity
        'semantic' AS source
    FROM spare_part_sample
    WHERE embedding IS NOT NULL
    ORDER BY embedding <=> %s::vector
    LIMIT 20
),

-- 2) KEYWORD SEARCH (BM25): Full-text search on tsv
keyword_search AS (
    SELECT 
        pk_plant_matnum,
        material_description,
        uom,
        ts_rank(tsv, plainto_tsquery('simple', %s)) AS bm25_score,
        'keyword' AS source
    FROM spare_part_sample
    WHERE tsv @@ plainto_tsquery('simple', %s)
    ORDER BY bm25_score DESC
    LIMIT 20
),

-- 3) EXACT CODE MATCH: Array overlap search (highest priority)
exact_code_match AS (
    SELECT 
        pk_plant_matnum,
        material_description,
        uom,
        CASE 
            WHEN keyword_codes && %s::TEXT[] THEN 1.0  -- exact match in identifier codes
            WHEN keyword_list && %s::TEXT[] THEN 0.7   -- match in keyword list
            ELSE 0.0
        END AS code_match_score,
        'exact_code' AS source
    FROM spare_part_sample
    WHERE keyword_codes && %s::TEXT[] OR keyword_list && %s::TEXT[]
),

-- 4) TRIGRAM FUZZY MATCH: Fuzzy string matching on pk
trigram_search AS (
    SELECT 
        pk_plant_matnum,
        material_description,
        uom,
        similarity(pk_plant_matnum, %s) AS trigram_score,
        'trigram' AS source
    FROM spare_part_sample
    WHERE pk_plant_matnum %% %s  -- trigram similarity operator
    ORDER BY trigram_score DESC
    LIMIT 10
),

-- 5) COMBINE ALL RESULTS
combined_results AS (
    SELECT pk_plant_matnum, material_description, uom,
           semantic_score, 0.0 AS bm25_score, 0.0 AS code_match_score, 0.0 AS trigram_score
    FROM semantic_search
    
    UNION ALL
    
    SELECT pk_plant_matnum, material_description, uom,
           0.0 AS semantic_score, bm25_score, 0.0 AS code_match_score, 0.0 AS trigram_score
    FROM keyword_search
    
    UNION ALL
    
    SELECT pk_plant_matnum, material_description, uom,
           0.0 AS semantic_score, 0.0 AS bm25_score, code_match_score, 0.0 AS trigram_score
    FROM exact_code_match
    
    UNION ALL
    
    SELECT pk_plant_matnum, material_description, uom,
           0.0 AS semantic_score, 0.0 AS bm25_score, 0.0 AS code_match_score, trigram_score
    FROM trigram_search
),

-- 6) AGGREGATE SCORES PER PK
aggregated_scores AS (
    SELECT 
        pk_plant_matnum,
        MAX(material_description) AS material_description,
        MAX(uom) AS uom,
        MAX(semantic_score) AS semantic_score,
        MAX(bm25_score) AS bm25_score,
        MAX(code_match_score) AS code_match_score,
        MAX(trigram_score) AS trigram_score
    FROM combined_results
    GROUP BY pk_plant_matnum
),

-- 7) NORMALIZE SCORES (Min-Max Scaling)
normalized_scores AS (
    SELECT 
        pk_plant_matnum,
        material_description,
        uom,
        -- Normalize each score to [0,1] range
        CASE 
            WHEN MAX(semantic_score) OVER () > 0 
            THEN semantic_score / MAX(semantic_score) OVER ()
            ELSE 0
        END AS sem_norm,
        CASE 
            WHEN MAX(bm25_score) OVER () > 0 
            THEN bm25_score / MAX(bm25_score) OVER ()
            ELSE 0
        END AS bm25_norm,
        code_match_score AS code_norm,  -- already [0,1]
        CASE 
            WHEN MAX(trigram_score) OVER () > 0 
            THEN trigram_score / MAX(trigram_score) OVER ()
            ELSE 0
        END AS trgm_norm,
        -- Keep raw scores for debugging
        semantic_score,
        bm25_score,
        code_match_score,
        trigram_score
    FROM aggregated_scores
)

-- 8) FINAL WEIGHTED SCORING & RANKING
SELECT 
    pk_plant_matnum,
    material_description,
    uom,
    
    -- WEIGHTED FINAL SCORE (adjust weights as needed)
    (
        10.0 * code_norm +           -- Exact code match (highest weight)
        5.0 * sem_norm +              -- Semantic similarity  
        3.0 * bm25_norm +             -- Keyword match (BM25)
        2.0 * trgm_norm               -- Trigram fuzzy match
    ) AS final_score,
    
    -- Individual normalized scores (for analysis)
    ROUND(code_norm::numeric, 3) AS code_score,
    ROUND(sem_norm::numeric, 3) AS semantic_score,
    ROUND(bm25_norm::numeric, 3) AS bm25_score,
    ROUND(trgm_norm::numeric, 3) AS trigram_score,
    
    -- Raw scores (for debugging)
    semantic_score AS raw_semantic,
    bm25_score AS raw_bm25,
    code_match_score AS raw_code,
    trigram_score AS raw_trigram

FROM normalized_scores
ORDER BY final_score DESC
LIMIT 30;

-- ============================================================
-- Parameters to bind:
-- %s -> query_embedding (vector)
-- %s -> query_embedding (vector) [duplicate for ORDER BY]
-- %s -> query_text (string for BM25)
-- %s -> query_text (string for BM25) [duplicate for WHERE]
-- %s -> extracted_codes (TEXT[] array)
-- %s -> extracted_codes (TEXT[] array) [duplicate]
-- %s -> extracted_codes (TEXT[] array) [for keyword_codes]
-- %s -> extracted_codes (TEXT[] array) [for keyword_list]
-- %s -> query_text (string for trigram)
-- %s -> query_text (string for trigram) [duplicate]
-- ============================================================
"""

### Field Weights Configuration

แนะนำการตั้งค่า weights สำหรับแต่ละ search method

In [54]:
# ============================================================
# Field Weights Configuration for Hybrid Search
# ============================================================

FIELD_WEIGHTS = {
    # -------------------------------------------------------
    # TIER 1: Exact Match (ความแม่นยำสูงสุด)
    # -------------------------------------------------------
    "exact_code_match": 10.0,          # keyword_codes array overlap
    "keyword_list_match": 7.0,         # keyword_list array overlap (รองลงมา)
    
    # -------------------------------------------------------
    # TIER 2: Semantic Understanding
    # -------------------------------------------------------
    "semantic_similarity": 5.0,        # embedding cosine similarity
    
    # -------------------------------------------------------
    # TIER 3: Keyword Relevance (BM25)
    # -------------------------------------------------------
    "bm25_full_text": 3.0,            # clean_combine_for_keyword (tsv)
    "uom_match_bonus": 4.0,            # ถ้า UOM ตรงกัน ให้ bonus
    
    # -------------------------------------------------------
    # TIER 4: Fuzzy Match
    # -------------------------------------------------------
    "trigram_similarity": 2.0,         # pk_plant_matnum fuzzy match
    
    # -------------------------------------------------------
    # Special Boosters (optional)
    # -------------------------------------------------------
    "same_plant_bonus": 1.0,           # material จาก plant เดียวกัน
    "recent_match_bonus": 0.5,         # recently added/updated
}

# ============================================================
# Recommended Weight Profiles for Different Use Cases
# ============================================================

WEIGHT_PROFILES = {
    # Profile 1: Precision-focused (เน้นความแม่นยำ)
    "precision": {
        "exact_code_match": 15.0,
        "semantic_similarity": 3.0,
        "bm25_full_text": 2.0,
        "trigram_similarity": 1.0,
    },
    
    # Profile 2: Balanced (สมดุลระหว่างความแม่นยำและความครอบคลุม)
    "balanced": {
        "exact_code_match": 10.0,
        "semantic_similarity": 5.0,
        "bm25_full_text": 3.0,
        "trigram_similarity": 2.0,
    },
    
    # Profile 3: Recall-focused (เน้นหาให้ครบ แม้จะไม่แม่นมาก)
    "recall": {
        "exact_code_match": 8.0,
        "semantic_similarity": 6.0,
        "bm25_full_text": 5.0,
        "trigram_similarity": 4.0,
    },
}

# ============================================================
# Usage in Query
# ============================================================
def calculate_final_score(
    code_score: float,
    semantic_score: float, 
    bm25_score: float,
    trigram_score: float,
    profile: str = "balanced"
) -> float:
    """
    Calculate weighted final score based on selected profile
    
    Args:
        code_score: Normalized exact code match score [0,1]
        semantic_score: Normalized semantic similarity [0,1]
        bm25_score: Normalized BM25 score [0,1]
        trigram_score: Normalized trigram similarity [0,1]
        profile: Weight profile to use ('precision', 'balanced', 'recall')
    
    Returns:
        Weighted final score
    """
    weights = WEIGHT_PROFILES.get(profile, WEIGHT_PROFILES["balanced"])
    
    return (
        weights.get("exact_code_match", 10.0) * code_score +
        weights.get("semantic_similarity", 5.0) * semantic_score +
        weights.get("bm25_full_text", 3.0) * bm25_score +
        weights.get("trigram_similarity", 2.0) * trigram_score
    )

# Example
print("Weight Profiles:")
for name, weights in WEIGHT_PROFILES.items():
    print(f"\n{name.upper()}:")
    for key, val in weights.items():
        print(f"  {key}: {val}")

Weight Profiles:

PRECISION:
  exact_code_match: 15.0
  semantic_similarity: 3.0
  bm25_full_text: 2.0
  trigram_similarity: 1.0

BALANCED:
  exact_code_match: 10.0
  semantic_similarity: 5.0
  bm25_full_text: 3.0
  trigram_similarity: 2.0

RECALL:
  exact_code_match: 8.0
  semantic_similarity: 6.0
  bm25_full_text: 5.0
  trigram_similarity: 4.0


## 🚀 Complete Hybrid Search Implementation

**Database**: `apm` | **Schema**: `apm_dev` | **Table**: `spare_part_test`

รวม logic:
1. ✅ Create table with all fields
2. ✅ Insert data + BGE-M3 embeddings
3. ✅ Hybrid search query with weighted scoring

In [55]:
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "black_kalareemhotmail.com",
    "password": "",
    "options": "-c search_path=apm_dev"  # Set default schema
}

In [56]:
import psycopg2
import psycopg2.extras
from pgvector.psycopg2 import register_vector
import numpy as np
from typing import List, Dict, Any, Optional, Tuple
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm

# ============================================================
# Configuration
# ============================================================
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "black_kalareemhotmail.com",
    "password": "",
    "options": "-c search_path=apm_dev,public -c client_encoding=UTF8"  # Set default schema and UTF-8 encoding
}

TABLE_NAME = "spare_part_test"
SCHEMA_NAME = "apm_dev"
EMBEDDING_DIM = 1024  # bge-m3 dimension

# ============================================================
# Database Connection
# ============================================================
def get_connection():
    """Get database connection with pgvector support"""
    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_client_encoding('UTF8')  # Ensure UTF-8 encoding for Thai text
    register_vector(conn)
    return conn

# ============================================================
# 1. CREATE TABLE
# ============================================================
def create_spare_part_table(drop_if_exists: bool = False):
    """
    Create spare_part_test table with all required fields
    
    Args:
        drop_if_exists: If True, drop existing table before creating
    """
    
    conn = get_connection()
    cur = conn.cursor()
    
    try:
        # Drop table if requested
        if drop_if_exists:
            print(f"⚠️  Dropping existing table {SCHEMA_NAME}.{TABLE_NAME}...")
            cur.execute(f"DROP TABLE IF EXISTS {SCHEMA_NAME}.{TABLE_NAME} CASCADE")
        
        # Create table
        create_sql = f"""
        CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.{TABLE_NAME} (
            -- Primary Key
            pk_plant_matnum TEXT PRIMARY KEY,
            
            -- Basic Info
            material_type TEXT,
            plant TEXT,
            material_number TEXT,
            plant_id TEXT,
            plant_description TEXT,
            uom TEXT,
            
            -- Raw Description Fields
            material_description TEXT,
            po_text TEXT,
            
            -- ============================================================
            -- SEMANTIC SEARCH Fields
            -- ============================================================
            raw_combine_for_embedding TEXT,
            clean_combine_for_embedding TEXT,
            embedding vector({EMBEDDING_DIM}),
            
            -- ============================================================
            -- KEYWORD SEARCH Fields (BM25)
            -- ============================================================
            raw_combine_for_keyword TEXT,
            clean_combine_for_keyword TEXT,
            tsv tsvector,
            
            -- Extracted Keywords/Codes
            keyword_codes TEXT[],
            keyword_old_codes TEXT[],
            keyword_list TEXT[],
            
            -- Metadata
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        
        print(f"📦 Creating table {SCHEMA_NAME}.{TABLE_NAME}...")
        cur.execute(create_sql)
        
        # Enable required extensions
        print("🔧 Ensuring extensions are enabled...")
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
        cur.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm")
        
        # Create indexes
        print("🔍 Creating indexes...")
        
        # A) Vector index for semantic search
        cur.execute(f"""
            DO $$
            BEGIN
                IF NOT EXISTS (
                    SELECT 1 FROM pg_indexes 
                    WHERE schemaname='{SCHEMA_NAME}' 
                    AND tablename='{TABLE_NAME}' 
                    AND indexname='idx_{TABLE_NAME}_embedding'
                ) THEN
                    CREATE INDEX idx_{TABLE_NAME}_embedding 
                    ON {SCHEMA_NAME}.{TABLE_NAME} 
                    USING ivfflat (embedding vector_cosine_ops)
                    WITH (lists = 100);
                END IF;
            END
            $$;
        """)
        
        # B) Full-text search index
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_tsv 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin(tsv);
        """)
        
        # C) Trigram index
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_pk_trgm 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (pk_plant_matnum gin_trgm_ops);
        """)
        
        # D) Array indexes for keyword matching
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_keyword_codes 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (keyword_codes);
            
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_keyword_list 
            ON {SCHEMA_NAME}.{TABLE_NAME} 
            USING gin (keyword_list);
        """)
        
        # E) Basic indexes
        cur.execute(f"""
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_material_type 
            ON {SCHEMA_NAME}.{TABLE_NAME} (material_type);
            
            CREATE INDEX IF NOT EXISTS idx_{TABLE_NAME}_uom 
            ON {SCHEMA_NAME}.{TABLE_NAME} (uom);
        """)
        
        # Create trigger for auto-updating tsv
        print("⚙️  Creating trigger for tsvector auto-update...")
        cur.execute(f"""
            CREATE OR REPLACE FUNCTION {SCHEMA_NAME}.{TABLE_NAME}_tsv_trigger() 
            RETURNS TRIGGER AS $$
            BEGIN
                NEW.tsv := to_tsvector('simple', COALESCE(NEW.clean_combine_for_keyword, ''));
                NEW.updated_at := CURRENT_TIMESTAMP;
                RETURN NEW;
            END;
            $$ LANGUAGE plpgsql;
            
            DROP TRIGGER IF EXISTS tsv_update_trigger ON {SCHEMA_NAME}.{TABLE_NAME};
            
            CREATE TRIGGER tsv_update_trigger 
            BEFORE INSERT OR UPDATE ON {SCHEMA_NAME}.{TABLE_NAME}
            FOR EACH ROW 
            EXECUTE FUNCTION {SCHEMA_NAME}.{TABLE_NAME}_tsv_trigger();
        """)
        
        conn.commit()
        print(f"✅ Table {SCHEMA_NAME}.{TABLE_NAME} created successfully!")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Error creating table: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Create Table")
print("="*60)
# get_connection()
create_spare_part_table(drop_if_exists=True)  # Uncomment to run

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TEST: Create Table
⚠️  Dropping existing table apm_dev.spare_part_test...
📦 Creating table apm_dev.spare_part_test...
🔧 Ensuring extensions are enabled...
🔍 Creating indexes...
⚙️  Creating trigger for tsvector auto-update...
✅ Table apm_dev.spare_part_test created successfully!


In [57]:
# ============================================================
# 2. EMBEDDER CLASS (BGE-M3)
# ============================================================
class BGE_M3_Embedder:
    """
    BGE-M3 Embedding Model Wrapper
    """
    def __init__(self, model_name: str = "BAAI/bge-m3", device: str = None):
        """
        Initialize BGE-M3 model
        
        Args:
            model_name: Model identifier (default: BAAI/bge-m3)
            device: 'cuda', 'cpu', or None (auto-detect)
        """
        print(f"🔄 Loading embedding model: {model_name}...")
        self.model = SentenceTransformer(model_name, device=device)
        self.dim = 1024  # bge-m3 output dimension
        print(f"✅ Model loaded! Dimension: {self.dim}")
    
    def embed_texts(
        self, 
        texts: List[str], 
        batch_size: int = 32,
        show_progress: bool = True
    ) -> List[List[float]]:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            batch_size: Batch size for encoding
            show_progress: Show progress bar
            
        Returns:
            List of embedding vectors (each is list of floats)
        """
        if not texts:
            return []
        
        embeddings = self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            convert_to_numpy=True,
            normalize_embeddings=True  # Recommended for bge-m3
        )
        
        return [emb.tolist() for emb in embeddings]
    
    def embed_single(self, text: str) -> List[float]:
        """Embed a single text (convenience method)"""
        return self.embed_texts([text], show_progress=False)[0]

# Test
print("="*60)
print("TEST: BGE-M3 Embedder")
print("="*60)
# embedder = BGE_M3_Embedder()  # Uncomment to test
# test_emb = embedder.embed_single("BEARING BALL TEST")
# print(f"Embedding dim: {len(test_emb)}")
# print(f"First 5 values: {test_emb[:5]}")

TEST: BGE-M3 Embedder


In [58]:
# ============================================================
# 3. INSERT DATA WITH EMBEDDINGS
# ============================================================
def insert_spare_parts(
    df: pd.DataFrame, 
    embedder: BGE_M3_Embedder, 
    batch_size: int = 128,
    start_from: int = 0
):
    """
    Insert spare part data into database with embeddings
    
    Required DataFrame columns:
        - pk_plant_matnum (PRIMARY KEY)
        - material_type, plant, material_number, plant_id, plant_description, uom
        - material_description, po_text
        - raw_combine_for_embedding, clean_combine_for_embedding
        - raw_combine_for_keyword, clean_combine_for_keyword
        - keyword_codes (list), keyword_old_codes (list), keyword_list (list)
    
    Args:
        df: DataFrame with spare part data
        embedder: BGE_M3_Embedder instance
        batch_size: Number of records to process per batch
        start_from: Skip first N records (for resuming)
    """
    
    conn = get_connection()
    cur = conn.cursor()
    
    # Normalize column names to lowercase
    df = df.copy()
    df.columns = df.columns.str.lower()
    
    # Required columns check
    required_cols = [
        'pk_plant_matnum', 'material_type', 'plant', 'material_number', 
        'plantid', 'plant_description', 'uom', 'material_description', 'text',
        'raw_combine_for_embedding', 'clean_combine_for_embedding',
        'raw_combine_for_keyword', 'clean_combine_for_keyword',
        'keyword_codes', 'keyword_old_codes', 'keyword_list'
    ]
    
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")
    
    # Prepare data
    df_to_insert = df[required_cols].copy()
    total_rows = len(df_to_insert)
    
    if start_from > 0:
        df_to_insert = df_to_insert.iloc[start_from:]
        print(f"⏭️  Skipping first {start_from} records...")
    
    print(f"📊 Total records to insert: {len(df_to_insert)}")
    
    insert_sql = f"""
        INSERT INTO {SCHEMA_NAME}.{TABLE_NAME} (
            pk_plant_matnum, material_type, plant, material_number,
            plant_id, plant_description, uom, material_description, po_text,
            raw_combine_for_embedding, clean_combine_for_embedding, embedding,
            raw_combine_for_keyword, clean_combine_for_keyword,
            keyword_codes, keyword_old_codes, keyword_list
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON CONFLICT (pk_plant_matnum) 
        DO UPDATE SET
            material_type = EXCLUDED.material_type,
            material_description = EXCLUDED.material_description,
            clean_combine_for_embedding = EXCLUDED.clean_combine_for_embedding,
            clean_combine_for_keyword = EXCLUDED.clean_combine_for_keyword,
            embedding = EXCLUDED.embedding,
            keyword_codes = EXCLUDED.keyword_codes,
            keyword_list = EXCLUDED.keyword_list,
            updated_at = CURRENT_TIMESTAMP
    """
    
    try:
        inserted = 0
        failed = 0
        
        # Process in batches
        for i in tqdm(range(0, len(df_to_insert), batch_size), desc="Inserting batches"):
            batch_df = df_to_insert.iloc[i:i+batch_size]
            
            # Generate embeddings for this batch
            texts_to_embed = batch_df['clean_combine_for_embedding'].fillna("").tolist()
            embeddings = embedder.embed_texts(texts_to_embed, show_progress=False)
            
            # Prepare batch data
            batch_data = []
            for idx, (_, row) in enumerate(batch_df.iterrows()):
                # Handle column name variations
                plant_id = row.get('plant_id') or row.get('plantid', '')
                po_text = row.get('po_text') or row.get('text', '')
                
                batch_data.append((
                    row['pk_plant_matnum'],
                    row['material_type'],
                    row['plant'],
                    row['material_number'],
                    plant_id,
                    row['plant_description'],
                    row['uom'],
                    row['material_description'],
                    po_text,
                    row['raw_combine_for_embedding'],
                    row['clean_combine_for_embedding'],
                    embeddings[idx],  # embedding vector
                    row['raw_combine_for_keyword'],
                    row['clean_combine_for_keyword'],
                    row['keyword_codes'] if isinstance(row['keyword_codes'], list) else [],
                    row['keyword_old_codes'] if isinstance(row['keyword_old_codes'], list) else [],
                    row['keyword_list'] if isinstance(row['keyword_list'], list) else []
                ))
            
            # Execute batch insert
            try:
                psycopg2.extras.execute_batch(cur, insert_sql, batch_data, page_size=batch_size)
                conn.commit()
                inserted += len(batch_data)
            except Exception as e:
                conn.rollback()
                failed += len(batch_data)
                print(f"\n❌ Batch {i//batch_size + 1} failed: {e}")
                continue
        
        print(f"\n✅ Insertion complete!")
        print(f"   Inserted: {inserted} records")
        if failed > 0:
            print(f"   Failed: {failed} records")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Error during insertion: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Insert Data")
print("="*60)
# Uncomment to run:
embedder = BGE_M3_Embedder()
insert_spare_parts(sample_100, embedder, batch_size=32)

TEST: Insert Data
🔄 Loading embedding model: BAAI/bge-m3...
✅ Model loaded! Dimension: 1024
📊 Total records to insert: 94


Inserting batches: 100%|██████████| 3/3 [00:04<00:00,  1.39s/it]


✅ Insertion complete!
   Inserted: 94 records


### 📚 Hybrid Search Methodology Summary

## 🎯 ภาพรวม Hybrid Search
ระบบใช้การรวม **4 วิธีการค้นหา** เพื่อหา material ที่คล้ายกัน:

---

## 🔍 วิธีการค้นหาทั้ง 4 แบบ

### 1️⃣ **Exact Code Match** (การจับคู่รหัสที่ตรงกัน)
- **วิธีการ**: ตรวจสอบว่า query มีรหัส (part code, drawing number) ที่ตรงกับ `keyword_codes` หรือ `keyword_list`
- **เทคนิค**: PostgreSQL Array Overlap (`&&` operator)
- **คะแนน**: 
  - 1.0 = ตรงกับ `keyword_codes` (รหัสหลัก)
  - 0.7 = ตรงกับ `keyword_list` (keyword ทั่วไป)
  - 0.0 = ไม่ตรง
- **จุดเด่น**: แม่นยำสูงมาก เหมาะกับการหารหัสที่ตรงกัน 100%

### 2️⃣ **Semantic Search** (การค้นหาตามความหมาย)
- **วิธีการ**: เปรียบเทียบ embedding vectors ระหว่าง query กับ records ใน database
- **Model**: BAAI/bge-m3 (1024 dimensions)
- **เทคนิค**: Cosine Similarity ผ่าน pgvector extension
- **คะแนน**: 0.0 - 1.0 (ยิ่งใกล้ 1 = ยิ่งคล้ายกัน)
- **จุดเด่น**: เข้าใจความหมาย เช่น "ball bearing" คล้ายกับ "rolling element bearing"

### 3️⃣ **BM25 Full-Text Search** (การค้นหาแบบคีย์เวิร์ด)
- **วิธีการ**: ค้นหาคำที่ตรงกันใน `clean_combine_for_keyword` โดยใช้ tsvector
- **เทคนิค**: PostgreSQL `ts_rank()` กับ `plainto_tsquery()`
- **คะแนน**: ขึ้นกับความถี่ของคำ และ term frequency
- **จุดเด่น**: เร็ว, ดีกับการค้นหาคำเฉพาะ (technical terms)

### 4️⃣ **Trigram Fuzzy Match** (การค้นหาแบบคล้ายๆ)
- **วิธีการ**: หาความคล้ายของ string ผ่าน trigram comparison บน `pk_plant_matnum`
- **เทคนิค**: PostgreSQL `pg_trgm` extension, `similarity()` function
- **คะแนน**: 0.0 - 1.0 (ยิ่งใกล้ 1 = string ยิ่งคล้ายกัน)
- **จุดเด่น**: รองรับ typo, เหมาะกับการหา material code ที่คล้ายๆ กัน

---

## ⚖️ Weight Profiles (น้ำหนักคะแนน)

### 🎯 **Precision Profile** (เน้นความแม่นยำ)
```python
{
    "exact_code_match": 15.0,      # ⭐ น้ำหนักสูงสุด
    "semantic_similarity": 3.0,
    "bm25_full_text": 2.0,
    "trigram_similarity": 1.0
}
```
- **เหมาะกับ**: ต้องการผลลัพธ์ที่แม่นมาก มี part code ตรงกัน
- **Use case**: หา material ที่ระบุ part number ชัดเจน

### ⚖️ **Balanced Profile** (สมดุล - **Default**)
```python
{
    "exact_code_match": 10.0,
    "semantic_similarity": 5.0,     # ⭐ เน้นความหมายมากขึ้น
    "bm25_full_text": 3.0,
    "trigram_similarity": 2.0
}
```
- **เหมาะกับ**: ใช้งานทั่วไป ต้องการทั้งความแม่นยำและความครอบคลุม
- **Use case**: ค้นหาแบบปกติ รองรับทั้งรหัสและคำอธิบาย

### 🌐 **Recall Profile** (เน้นความครอบคลุม)
```python
{
    "exact_code_match": 8.0,
    "semantic_similarity": 6.0,     # ⭐ เน้นความหมายสูงสุด
    "bm25_full_text": 5.0,          # ⭐ เน้น keyword matching
    "trigram_similarity": 4.0       # ⭐ เน้น fuzzy matching
}
```
- **เหมาะกับ**: ต้องการหาผลลัพธ์ให้มากที่สุด ยอมให้ผลลัพธ์บางตัวไม่ตรง
- **Use case**: exploratory search, หา material ที่อาจจะเกี่ยวข้อง

---

## 🧮 วิธีคำนวณ Final Score

```
Final Score = (W_code × Code_normalized) + 
              (W_sem × Semantic_normalized) + 
              (W_bm25 × BM25_normalized) + 
              (W_trigram × Trigram_normalized)
```

**ขั้นตอน**:
1. แต่ละวิธีค้นหาได้คะแนนดิบ (raw score)
2. **Normalize** คะแนนแต่ละวิธีให้อยู่ในช่วง [0, 1] โดยหารด้วย max score
3. คูณด้วย weight ตาม profile ที่เลือก
4. รวมคะแนนทั้งหมด = Final Score
5. เรียงตาม Final Score จากมากไปน้อย

---

## 💡 คำแนะนำในการปรับ Weight

- **เพิ่มน้ำหนัก Code Match** → ถ้าต้องการให้ part code มีความสำคัญมากที่สุด
- **เพิ่มน้ำหนัก Semantic** → ถ้า description มีความสำคัญ และต้องการเข้าใจความหมาย
- **เพิ่มน้ำหนัก BM25** → ถ้าต้องการจับคำเฉพาะ (specific technical terms)
- **เพิ่มน้ำหนัก Trigram** → ถ้ามี typo หรือต้องการรองรับ fuzzy matching

---

## 🚀 Use Case ตามที่อธิบาย

**โจทย์**: วน loop ทุก record เพื่อหา candidates ที่อาจเป็น material เดียวกัน แล้วส่งให้ LLM ประเมิน

**วิธีใช้**:
```python
for idx, record in df.iterrows():
    query_text = record['clean_combine_for_embedding']
    
    # หา top candidates
    candidates = hybrid_search(
        query_text=query_text,
        embedder=embedder,
        weight_profile="balanced",  # หรือ precision/recall
        limit=10  # เอา top 10 candidates
    )
    
    # ส่งให้ LLM ประเมิน
    # main_record = record
    # candidate_list = candidates
    # → LLM ตัดสินว่าตัวไหนเป็น duplicate
```

In [59]:
# ============================================================
# 4. HYBRID SEARCH QUERY
# ============================================================

# Weight Profiles
WEIGHT_PROFILES = {
    "precision": {
        "exact_code_match": 15.0,
        "semantic_similarity": 3.0,
        "bm25_full_text": 2.0,
        "trigram_similarity": 1.0
    },
    "balanced": {
        "exact_code_match": 10.0,
        "semantic_similarity": 5.0,
        "bm25_full_text": 3.0,
        "trigram_similarity": 2.0
    },
    "recall": {
        "exact_code_match": 8.0,
        "semantic_similarity": 6.0,
        "bm25_full_text": 5.0,
        "trigram_similarity": 4.0
    }
}

def hybrid_search(
    query_text: str,
    embedder: BGE_M3_Embedder,
    weight_profile: str = "balanced",
    top_k_semantic: int = 20,
    top_k_bm25: int = 20,
    top_k_trigram: int = 10,
    limit: int = 30,
    uom_filter: Optional[str] = None,
    material_type_filter: Optional[str] = None
) -> pd.DataFrame:
    """
    Perform hybrid search combining semantic, keyword, and trigram matching
    
    Args:
        query_text: Search query text
        embedder: BGE_M3_Embedder instance
        weight_profile: 'precision', 'balanced', or 'recall'
        top_k_semantic: Top K results from semantic search
        top_k_bm25: Top K results from BM25
        top_k_trigram: Top K results from trigram
        limit: Final result limit
        uom_filter: Optional UOM filter (e.g., 'PC')
        material_type_filter: Optional material type filter
        
    Returns:
        DataFrame with search results and scores
    """
    
    # Get weight profile
    weights = WEIGHT_PROFILES.get(weight_profile, WEIGHT_PROFILES["balanced"])
    w_code = weights["exact_code_match"]
    w_sem = weights["semantic_similarity"]
    w_bm25 = weights["bm25_full_text"]
    w_trgm = weights["trigram_similarity"]
    
    # Generate query embedding
    print(f"🔄 Generating query embedding...")
    query_embedding = embedder.embed_single(query_text)
    
    # Extract potential codes from query
    query_codes = extract_identifier_keywords(query_text.upper())
    
    print(f"🔍 Searching with profile: {weight_profile}")
    print(f"   Weights: Code={w_code}, Semantic={w_sem}, BM25={w_bm25}, Trigram={w_trgm}")
    print(f"   Extracted codes from query: {query_codes}")
    
    conn = get_connection()
    cur = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    
    try:
        # Build filters
        filter_clause = ""
        if uom_filter:
            filter_clause += f" AND uom = '{uom_filter}'"
        if material_type_filter:
            filter_clause += f" AND material_type = '{material_type_filter}'"
        
        # Execute hybrid search query
        search_sql = f"""
        WITH 
        -- 1) SEMANTIC SEARCH
        semantic_search AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                (1 - (embedding <=> %s::vector)) AS semantic_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE embedding IS NOT NULL {filter_clause}
            ORDER BY embedding <=> %s::vector
            LIMIT {top_k_semantic}
        ),
        
        -- 2) KEYWORD SEARCH (BM25)
        keyword_search AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                ts_rank(tsv, plainto_tsquery('simple', %s)) AS bm25_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE tsv @@ plainto_tsquery('simple', %s) {filter_clause}
            ORDER BY bm25_score DESC
            LIMIT {top_k_bm25}
        ),
        
        -- 3) EXACT CODE MATCH
        exact_code_match AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                CASE 
                    WHEN keyword_codes && %s::TEXT[] THEN 1.0
                    WHEN keyword_list && %s::TEXT[] THEN 0.7
                    ELSE 0.0
                END AS code_match_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE (keyword_codes && %s::TEXT[] OR keyword_list && %s::TEXT[]) {filter_clause}
        ),
        
        -- 4) TRIGRAM FUZZY MATCH
        trigram_search AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                similarity(pk_plant_matnum, %s) AS trigram_score
            FROM {SCHEMA_NAME}.{TABLE_NAME}
            WHERE pk_plant_matnum %% %s {filter_clause}
            ORDER BY trigram_score DESC
            LIMIT {top_k_trigram}
        ),
        
        -- 5) COMBINE RESULTS
        combined_results AS (
            SELECT pk_plant_matnum, material_description, uom,
                   semantic_score, 0.0 AS bm25_score, 0.0 AS code_match_score, 0.0 AS trigram_score
            FROM semantic_search
            UNION ALL
            SELECT pk_plant_matnum, material_description, uom,
                   0.0, bm25_score, 0.0, 0.0
            FROM keyword_search
            UNION ALL
            SELECT pk_plant_matnum, material_description, uom,
                   0.0, 0.0, code_match_score, 0.0
            FROM exact_code_match
            UNION ALL
            SELECT pk_plant_matnum, material_description, uom,
                   0.0, 0.0, 0.0, trigram_score
            FROM trigram_search
        ),
        
        -- 6) AGGREGATE SCORES
        aggregated_scores AS (
            SELECT 
                pk_plant_matnum,
                MAX(material_description) AS material_description,
                MAX(uom) AS uom,
                MAX(semantic_score) AS semantic_score,
                MAX(bm25_score) AS bm25_score,
                MAX(code_match_score) AS code_match_score,
                MAX(trigram_score) AS trigram_score
            FROM combined_results
            GROUP BY pk_plant_matnum
        ),
        
        -- 7) NORMALIZE SCORES
        normalized_scores AS (
            SELECT 
                pk_plant_matnum,
                material_description,
                uom,
                CASE WHEN MAX(semantic_score) OVER () > 0 
                     THEN semantic_score / NULLIF(MAX(semantic_score) OVER (), 0)
                     ELSE 0 END AS sem_norm,
                CASE WHEN MAX(bm25_score) OVER () > 0 
                     THEN bm25_score / NULLIF(MAX(bm25_score) OVER (), 0)
                     ELSE 0 END AS bm25_norm,
                code_match_score AS code_norm,
                CASE WHEN MAX(trigram_score) OVER () > 0 
                     THEN trigram_score / NULLIF(MAX(trigram_score) OVER (), 0)
                     ELSE 0 END AS trgm_norm,
                semantic_score, bm25_score, code_match_score, trigram_score
            FROM aggregated_scores
        )
        
        -- 8) FINAL WEIGHTED SCORING
        SELECT 
            pk_plant_matnum,
            material_description,
            uom,
            ({w_code} * code_norm + {w_sem} * sem_norm + 
             {w_bm25} * bm25_norm + {w_trgm} * trgm_norm) AS final_score,
            ROUND(code_norm::numeric, 3) AS code_score,
            ROUND(sem_norm::numeric, 3) AS semantic_score,
            ROUND(bm25_norm::numeric, 3) AS bm25_score,
            ROUND(trgm_norm::numeric, 3) AS trigram_score
        FROM normalized_scores
        ORDER BY final_score DESC
        LIMIT {limit};
        """
        
        # Execute query
        cur.execute(search_sql, (
            query_embedding, query_embedding,  # semantic
            query_text, query_text,            # bm25
            query_codes, query_codes, query_codes, query_codes,  # exact code
            query_text, query_text             # trigram
        ))
        
        results = cur.fetchall()
        
        # Convert to DataFrame
        df_results = pd.DataFrame(results)
        
        print(f"✅ Found {len(df_results)} results")
        
        return df_results
        
    except Exception as e:
        print(f"❌ Search error: {e}")
        raise
    finally:
        cur.close()
        conn.close()

# Test
print("="*60)
print("TEST: Hybrid Search")
print("="*60)
# Uncomment to test with real data:
embedder = BGE_M3_Embedder()

# Get first record from sample_100 as query
query_record = sample_100.iloc[0]

# Handle column name case variations
def get_col(record, col_name):
    """Get column value regardless of case"""
    col_lower = col_name.lower()
    col_title = col_name.replace('_', ' ').title().replace(' ', '_')
    return record.get(col_lower, record.get(col_title, record.get(col_name, '')))

pk = get_col(query_record, 'pk_plant_matnum')
material_desc = get_col(query_record, 'Material_Description')
uom = get_col(query_record, 'UOM')
query_text = get_col(query_record, 'clean_combine_for_embedding')

print(f"\n{'='*80}")
print("🔍 QUERY RECORD (Record #1 from sample_100)")
print(f"{'='*80}")
print(f"PK: {pk}")
print(f"Material: {material_desc}")
print(f"UOM: {uom}")
print(f"Query Text: {query_text[:200]}...")

# Search for candidates
results = hybrid_search(
    query_text=query_text,
    embedder=embedder,
    weight_profile="balanced",
    limit=20
)

print(f"\n{'='*80}")
print("📊 TOP CANDIDATES (Sorted by Final Score)")
print(f"{'='*80}")
print(results[["pk_plant_matnum", "material_description", "uom", 
               "final_score", "code_score", "semantic_score", 
               "bm25_score", "trigram_score"]].to_string(index=False))

# Show top 5 in detail
print(f"\n{'='*80}")
print("🎯 TOP 5 CANDIDATES - DETAILED VIEW")
print(f"{'='*80}")
for idx, row in results.head(5).iterrows():
    print(f"\n[Rank {idx+1}] Final Score: {row['final_score']:.3f}")
    print(f"  PK: {row['pk_plant_matnum']}")
    print(f"  Description: {row['material_description']}")
    print(f"  UOM: {row['uom']}")
    print(f"  Score Breakdown:")
    print(f"    - Code Match:  {row['code_score']:.3f}")
    print(f"    - Semantic:    {row['semantic_score']:.3f}")
    print(f"    - BM25:        {row['bm25_score']:.3f}")
    print(f"    - Trigram:     {row['trigram_score']:.3f}")

TEST: Hybrid Search
🔄 Loading embedding model: BAAI/bge-m3...
✅ Model loaded! Dimension: 1024

🔍 QUERY RECORD (Record #1 from sample_100)
PK: 7560_75060602310346
Material: BEARING,ANGULAR CONTACT,3310 ATN9  [VMI]
UOM: PC
Query Text: short_description: bearing,angular contact,3310 atn9 [vmi] long_description: bearing, bore type : cylindrical, internal clearance : normal clearance, seal/shield type : no seal, type : angular contact...
🔄 Generating query embedding...
🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: ['T104A24']
✅ Found 20 results

📊 TOP CANDIDATES (Sorted by Final Score)
    pk_plant_matnum                     material_description uom  final_score code_score semantic_score bm25_score trigram_score
7560_75060602310346 BEARING,ANGULAR CONTACT,3310 ATN9  [VMI]  PC    15.000000      1.000          1.000      0.000         0.000
7560_75060602411346   BEARING,ANGULAR CONTACT,3311 ANR   [F]  PC     4.7745

In [ ]:
sample_100

,pk_plant_matnum,Material_Type,Plant,Material_Number,PlantID,Plant_Description,UOM,Material,Material_Description,Text,raw_combine_for_embedding,raw_combine_for_keyword,clean_combine_for_embedding,clean_combine_for_keyword,keyword_codes,keyword_old_codes,keyword_list
0,7560_75060602310346,6.0,7560,75060602310346,PT011,SKIC-Wangsa-la,PC,75060602310346,"BEARING,ANGULAR CONTACT,3310 ATN9 [VMI]","BEARING, BORE TYPE : CYLINDRICAL, INTERNAL CL...","short_description: BEARING,ANGULAR CONTACT,331...","BEARING,ANGULAR CONTACT,3310 ATN9 [VMI] BEARI...","short_description: bearing,angular contact,331...","bearing , angular contact , 3310 atn9 [vmi] be...",[T104A24],"[t104a24-b, u6b60-b, atn9]","[(tkic#6, 3310, 7), T104A24, [vmi], angular, a..."
1,7560_75060602411346,6.0,7560,75060602411346,PT011,SKIC-Wangsa-la,PC,75060602411346,"BEARING,ANGULAR CONTACT,3311 ANR [F]","BEARING, BORE TYPE : CYLINDRICAL, INTERNAL CL...","short_description: BEARING,ANGULAR CONTACT,331...","BEARING,ANGULAR CONTACT,3311 ANR [F] BEARING...","short_description: bearing,angular contact,331...","bearing , angular contact , 3311 anr [f] beari...",[T10A24],"[u6b60-b, t10a24-b]","[(tkic#6, 3311, 7), T10A24, [f], angular, anr,..."
2,7511_75060610300414,6.0,7511,75060610300414,PT010,SKIC-Ban Pong,PC,75060610300414,"BEARING,ANGULAR CONTACT,3310 ATN9(F)","BEARING, BORE TYPE : CYLINDRICAL, INTERNAL CL...","short_description: BEARING,ANGULAR CONTACT,331...","BEARING,ANGULAR CONTACT,3310 ATN9(F) BEARING, ...","short_description: bearing,angular contact,331...","bearing , angular contact , 3310 atn9(f) beari...",[],[atn9],"[3310, angular, atn9, atn9(f), bearing, bore, ..."
3,7511_75060610310502,6.0,7511,75060610310502,PT010,SKIC-Ban Pong,PC,75060610310502,"BEARING,ANGULAR CONTACT,3311 ANR(F)","BEARING, SHORT NAME : BEARING, BORE TYPE : C...","short_description: BEARING,ANGULAR CONTACT,331...","BEARING,ANGULAR CONTACT,3311 ANR(F) BEARING, S...","short_description: bearing,angular contact,331...","bearing , angular contact , 3311 anr(f) bearin...",[],[],"[#5, 3311, angular, anr, anr(f), ball, bearing..."
4,7511_75060610805312,6.0,7511,75060610805312,PT010,SKIC-Ban Pong,PC,75060610805312,"BEARING,ANGULAR CONTACT,7306 BECBM(F)","BEARING, BORE TYPE : CYLINDRICAL, INTERNAL CL...","short_description: BEARING,ANGULAR CONTACT,730...","BEARING,ANGULAR CONTACT,7306 BECBM(F) BEARING,...","short_description: bearing,angular contact,730...","bearing , angular contact , 7306 becbm(f) bear...",[7306-BECBM],[],"[7306, 7306-BECBM, 7306-becbm, angular, bearin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,LX11_LX06600-0238-1,6.0,LX11,LX06600-0238-1,PT028,UPPC-Philippines,PC,LX06600-0238-1,STAUFF SUCTION FLTER ELEMENT PN 58125E,STAUFF SUCTION FLTER ELEMENT PN 58125E FUNCTIO...,short_description: STAUFF SUCTION FLTER ELEMEN...,STAUFF SUCTION FLTER ELEMENT PN 58125E STAUFF ...,short_description: stauff suction flter elemen...,stauff suction flter element pn 58125e stauff ...,"[7LYP11-PM2-LH-304101-M-TK, PN 58125E]","[lyp11-pm2-lh-304101-m-tk, 8125e, pm2, 125um]","[58125e, 7-23-14, 7LYP11-PM2-LH-304101-M-TK, 7..."
90,7811_78067453720932,6.0,7811,78067453720932,PT014,TPC-Ban Pong,PC,78067453720932,"PP,SHAFT,1,AHLSTROM(ZN)HOLD","SHORT NAME : PP, MFR MODEL NO. : SF200877K1 ...","short_description: PP,SHAFT,1,AHLSTROM(ZN)HOLD...","PP,SHAFT,1,AHLSTROM(ZN)HOLD SHORT NAME : PP, ...","short_description: pp,shaft,1,ahlstrom(zn)hold...","pp , shaft , 1 , ahlstrom(zn)hold short name p...","[SF200869K1, SF200877K1, SFR00021K1]","[sf200877k1, sf200869k1, h5, sfr00021k1]","[1, SF200869K1, SF200877K1, SFR00021K1, ahlstr..."
91,LX11_LX06336-0324,6.0,LX11,LX06336-0324,PT028,UPPC-Philippines,PC,LX06336-0324,"""BEARING, 23236 CCK/W33""","BEARING, 23236 CCK/W33","short_description: ""BEARING, 23236 CCK/W33"" lo...","""BEARING, 23236 CCK/W33"" BEARING, 23236 CCK/W3...","short_description: ""bearing, 23236 cck/w33"" lo...","bearing , 23236 cck/w33 bearing , 23236 cck/w3...",[CCK/W33],[w33]

### 🧪 Evaluation: Find Answer Rank in Candidates

ประเมินว่าเฉลยอยู่ที่ rank เท่าไหร่ใน candidates จาก hybrid search

In [60]:
# ============================================================
# EVALUATION: CHECK ANSWER RANK IN CANDIDATES
# ============================================================

def evaluate_hybrid_search_quality(
    edges_df,
    sample_df,
    embedder,
    weight_profile="balanced",
    top_k=30
):
    """
    ประเมินคุณภาพของ hybrid search โดยเช็คว่าเฉลยอยู่ที่ rank เท่าไหร่
    
    Args:
        edges_df: DataFrame ที่มี Key_mat_1, Key_mat_2 (คู่ที่เป็นเฉลย)
        sample_df: DataFrame ที่มีข้อมูล material ทั้งหมด (sample_100)
        embedder: BGE_M3_Embedder instance
        weight_profile: 'precision', 'balanced', or 'recall'
        top_k: จำนวน candidates สูงสุดที่จะค้นหา
    
    Returns:
        DataFrame สรุปผลการประเมิน
    """
    
    # สร้าง mapping: pk -> คู่ของมัน (เฉลย)
    answer_map = {}
    for _, row in edges_df.iterrows():
        key1 = row['Key_mat_1']
        key2 = row['Key_mat_2']
        
        # ทั้งสองทิศทาง
        if pd.notna(key1) and pd.notna(key2):
            if key1 not in answer_map:
                answer_map[key1] = []
            if key2 not in answer_map:
                answer_map[key2] = []
            answer_map[key1].append(key2)
            answer_map[key2].append(key1)
    
    print(f"📊 Total PKs with answers: {len(answer_map)}")
    print(f"🔍 Starting evaluation with profile: {weight_profile}")
    print(f"⏱️  This may take a few minutes...\n")
    
    results = []
    
    # วน loop ทุก PK ที่มีเฉลย
    for pk, answers in tqdm(answer_map.items(), desc="Evaluating PKs"):
        # หา record นี้ใน sample_df
        record_rows = sample_df[sample_df['pk_plant_matnum'] == pk]
        
        if len(record_rows) == 0:
            continue  # skip ถ้าไม่มีใน sample
        
        record = record_rows.iloc[0]
        
        # Handle column case variations
        def get_col(rec, col_name):
            col_lower = col_name.lower()
            for col in rec.index:
                if col.lower() == col_lower:
                    return rec[col]
            return ''
        
        query_text = get_col(record, 'clean_combine_for_embedding')
        material_desc = get_col(record, 'material_description')
        
        if not query_text or pd.isna(query_text) or query_text == '':
            continue
        
        try:
            # ทำ hybrid search
            candidates = hybrid_search(
                query_text=query_text,
                embedder=embedder,
                weight_profile=weight_profile,
                limit=top_k
            )
            
            # เช็คว่าเฉลยอยู่ที่ rank ไหน
            candidate_pks = candidates['pk_plant_matnum'].tolist()
            
            found_ranks = []
            for answer_pk in answers:
                if answer_pk in candidate_pks:
                    rank = candidate_pks.index(answer_pk) + 1  # rank เริ่มจาก 1
                    found_ranks.append(rank)
            
            # บันทึกผล
            if found_ranks:
                best_rank = min(found_ranks)
                status = "✅ Found"
            else:
                best_rank = None
                status = "❌ Not Found"
            
            results.append({
                'pk': pk,
                'material_description': material_desc[:50] if material_desc else '',
                'num_answers': len(answers),
                'answers': ', '.join(answers),
                'found_ranks': ', '.join(map(str, found_ranks)) if found_ranks else 'N/A',
                'best_rank': best_rank,
                'status': status
            })
            
        except Exception as e:
            print(f"\n⚠️  Error processing {pk}: {e}")
            continue
    
    # สร้าง DataFrame ผลลัพธ์
    results_df = pd.DataFrame(results)
    
    return results_df

# Test evaluation
print("="*80)
print("EVALUATION: Hybrid Search Quality Assessment")
print("="*80)
print("\nThis will evaluate all 34 PKs from df_40 that have answers in edges")
print("It will check at which rank the correct answer appears in candidates\n")

# Uncomment to run:
embedder = BGE_M3_Embedder()  # Load model if not already loaded

evaluation_results = evaluate_hybrid_search_quality(
    edges_df=sample20,
    sample_df=sample_100,
    embedder=embedder,
    weight_profile="balanced",
    top_k=30
)

# แสดงผลลัพธ์
print("\n" + "="*80)
print("📊 EVALUATION RESULTS")
print("="*80)
print(evaluation_results[['pk', 'material_description', 'best_rank', 'status']].to_string(index=False))

# สรุปสถิติ
print("\n" + "="*80)
print("📈 SUMMARY STATISTICS")
print("="*80)

total = len(evaluation_results)
found = len(evaluation_results[evaluation_results['status'] == '✅ Found'])
not_found = total - found

print(f"Total PKs evaluated: {total}")
print(f"Found in candidates: {found} ({found/total*100:.1f}%)")
print(f"Not found: {not_found} ({not_found/total*100:.1f}%)")

if found > 0:
    found_df = evaluation_results[evaluation_results['status'] == '✅ Found']
    print(f"\nRank Statistics (for found answers):")
    print(f"  Mean Rank: {found_df['best_rank'].mean():.2f}")
    print(f"  Median Rank: {found_df['best_rank'].median():.1f}")
    print(f"  Best Rank: {found_df['best_rank'].min()}")
    print(f"  Worst Rank: {found_df['best_rank'].max()}")
    
    # Rank distribution
    print(f"\n📊 Rank Distribution:")
    print(f"  Top 1: {len(found_df[found_df['best_rank'] == 1])}")
    print(f"  Top 3: {len(found_df[found_df['best_rank'] <= 3])}")
    print(f"  Top 5: {len(found_df[found_df['best_rank'] <= 5])}")
    print(f"  Top 10: {len(found_df[found_df['best_rank'] <= 10])}")
    print(f"  Top 20: {len(found_df[found_df['best_rank'] <= 20])}")

# Show cases where answer was not found
print("\n" + "="*80)
print("❌ CASES WHERE ANSWER NOT FOUND")
print("="*80)
not_found_df = evaluation_results[evaluation_results['status'] == '❌ Not Found']
if len(not_found_df) > 0:
    print(not_found_df[['pk', 'material_description', 'answers']].to_string(index=False))
else:
    print("🎉 All answers were found in candidates!")

EVALUATION: Hybrid Search Quality Assessment

This will evaluate all 34 PKs from df_40 that have answers in edges
It will check at which rank the correct answer appears in candidates

🔄 Loading embedding model: BAAI/bge-m3...
✅ Model loaded! Dimension: 1024
📊 Total PKs with answers: 34
🔍 Starting evaluation with profile: balanced
⏱️  This may take a few minutes...



Evaluating PKs:   0%|          | 0/34 [00:00<?, ?it/s]

🔄 Generating query embedding...


Evaluating PKs:   3%|▎         | 1/34 [00:00<00:13,  2.41it/s]

🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: ['T104A24']
✅ Found 20 results
🔄 Generating query embedding...
🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: []
✅ Found 20 results
🔄 Generating query embedding...
🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: ['T10A24']


...

🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: ['NUP-2206', 'NUP2206']
✅ Found 20 results
🔄 Generating query embedding...
🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: ['NUP-2206', 'NUP2206']
✅ Found 20 results
🔄 Generating query embedding...
🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: ['NUP2206ECP']
✅ Found 20 results
🔄 Generating query embedding...


Evaluating PKs:  91%|█████████ | 31/34 [00:02<00:00, 13.63it/s]

🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: ['D-75', 'NU-1015']
✅ Found 20 results
🔄 Generating query embedding...
🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: ['NU-1015']
✅ Found 20 results
🔄 Generating query embedding...
🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: []
✅ Found 20 results
🔄 Generating query embedding...


Evaluating PKs: 100%|██████████| 34/34 [00:03<00:00, 11.13it/s]

🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: []
✅ Found 20 results
🔄 Generating query embedding...
🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: []
✅ Found 20 results
🔄 Generating query embedding...
🔍 Searching with profile: balanced
   Weights: Code=10.0, Semantic=5.0, BM25=3.0, Trigram=2.0
   Extracted codes from query: []
✅ Found 20 results

📊 EVALUATION RESULTS
                 pk                     material_description  best_rank  status
7560_75060602310346 BEARING,ANGULAR CONTACT,3310 ATN9  [VMI]          3 ✅ Found
7511_75060610300414     BEARING,ANGULAR CONTACT,3310 ATN9(F)          3 ✅ Found
7560_75060602411346   BEARING,ANGULAR CONTACT,3311 ANR   [F]          3 ✅ Found
7511_75060610310502      BEARING,ANGULAR CONTACT,3311 ANR(F)          3 ✅ Found
7812_78060606306581 BEARING,ANGULAR CONTACT,7306 BECBM [VMI]          2 ✅

In [ ]:
# ต่อ 
# 1. insert data ทั้งหมดของ mat type 06 65k record ลง DB 
# 2. test กับ edges คำตอบ ทั้งหมด 1k record ดูว่า เฉลยอยู่ใน candidate ทั้งหมดไหม 
# 3. อย่าลืมเพิ่ม log time ที่ใช้ พร้อมกับ ลบ log ที่ไม่ใช้ 
# 4. อยากลอง test กับ ผลตัว balance / precision / recall



### 🎯 Quick Start Guide

วิธีใช้งานแบบ Step-by-Step

In [ ]:
# ============================================================
# COMPLETE WORKFLOW EXAMPLE
# ============================================================

# STEP 1: Create table (run once)
print("\n" + "="*60)
print("STEP 1: Create Table")
print("="*60)
create_spare_part_table(drop_if_exists=True)  # Set False if table exists

# STEP 2: Load embedder
print("\n" + "="*60)
print("STEP 2: Load BGE-M3 Model")
print("="*60)
embedder = BGE_M3_Embedder()

# STEP 3: Insert data
print("\n" + "="*60)
print("STEP 3: Insert Data with Embeddings")
print("="*60)

# Make sure your DataFrame has all required columns
# Example: use sample_100 or select_select_mm_matType
insert_spare_parts(
    df=sample_100,  # Replace with your DataFrame
    embedder=embedder,
    batch_size=32
)

# STEP 4: Verify data
print("\n" + "="*60)
print("STEP 4: Verify Data")
print("="*60)
conn = get_connection()
cur = conn.cursor()
cur.execute(f"SELECT COUNT(*) FROM {SCHEMA_NAME}.{TABLE_NAME}")
count = cur.fetchone()[0]
print(f"✅ Total records in table: {count}")
cur.close()
conn.close()

# STEP 5: Test search
print("\n" + "="*60)
print("STEP 5: Test Hybrid Search")
print("="*60)

# Example search
results = hybrid_search(
    query_text="BEARING BALL SKF",
    embedder=embedder,
    weight_profile="balanced",  # or 'precision' or 'recall'
    limit=10
)

print("\n📊 Search Results:")
print(results[["pk_plant_matnum", "material_description", "uom", "final_score"]].to_string())

print("\n🎉 Setup Complete!")

### 🔧 Utility Functions

Helper functions สำหรับการจัดการและ debug

In [ ]:
# ============================================================
# UTILITY FUNCTIONS
# ============================================================

def get_table_stats():
    """Get table statistics"""
    conn = get_connection()
    cur = conn.cursor()
    
    stats = {}
    
    # Total records
    cur.execute(f"SELECT COUNT(*) FROM {SCHEMA_NAME}.{TABLE_NAME}")
    stats['total_records'] = cur.fetchone()[0]
    
    # Records with embeddings
    cur.execute(f"SELECT COUNT(*) FROM {SCHEMA_NAME}.{TABLE_NAME} WHERE embedding IS NOT NULL")
    stats['with_embeddings'] = cur.fetchone()[0]
    
    # Unique UOMs
    cur.execute(f"SELECT COUNT(DISTINCT uom) FROM {SCHEMA_NAME}.{TABLE_NAME}")
    stats['unique_uoms'] = cur.fetchone()[0]
    
    # Unique material types
    cur.execute(f"SELECT COUNT(DISTINCT material_type) FROM {SCHEMA_NAME}.{TABLE_NAME}")
    stats['unique_material_types'] = cur.fetchone()[0]
    
    cur.close()
    conn.close()
    
    print("📊 Table Statistics:")
    for key, val in stats.items():
        print(f"   {key}: {val}")
    
    return stats

def get_sample_records(limit: int = 5):
    """Get sample records from table"""
    conn = get_connection()
    
    query = f"""
        SELECT pk_plant_matnum, material_description, uom, 
               array_length(keyword_codes, 1) as num_codes,
               embedding IS NOT NULL as has_embedding
        FROM {SCHEMA_NAME}.{TABLE_NAME}
        LIMIT {limit}
    """
    
    df = pd.read_sql(query, conn)
    conn.close()
    
    return df

def delete_all_records():
    """Delete all records from table (use with caution!)"""
    response = input(f"⚠️  Are you sure you want to delete ALL records from {SCHEMA_NAME}.{TABLE_NAME}? (yes/no): ")
    
    if response.lower() != 'yes':
        print("❌ Cancelled")
        return
    
    conn = get_connection()
    cur = conn.cursor()
    
    cur.execute(f"DELETE FROM {SCHEMA_NAME}.{TABLE_NAME}")
    deleted = cur.rowcount
    conn.commit()
    
    cur.close()
    conn.close()
    
    print(f"✅ Deleted {deleted} records")

def compare_weight_profiles(
    query_text: str,
    embedder: BGE_M3_Embedder,
    limit: int = 5
):
    """
    Compare search results across different weight profiles
    """
    profiles = ["precision", "balanced", "recall"]
    
    print(f"🔍 Comparing weight profiles for query: '{query_text}'\n")
    
    all_results = {}
    
    for profile in profiles:
        print(f"Running {profile} profile...")
        results = hybrid_search(
            query_text=query_text,
            embedder=embedder,
            weight_profile=profile,
            limit=limit
        )
        all_results[profile] = results
    
    # Display comparison
    print("\n" + "="*80)
    print("RESULTS COMPARISON")
    print("="*80)
    
    for profile in profiles:
        print(f"\n🎯 {profile.upper()} Profile:")
        df = all_results[profile]
        print(df[["pk_plant_matnum", "material_description", "final_score"]].to_string(index=False))
    
    return all_results

# Example usage
print("="*60)
print("UTILITY FUNCTIONS")
print("="*60)
# get_table_stats()
# print("\nSample records:")
# print(get_sample_records(limit=3))